In [ ]:
#DETECTOR DE ANOMALIAS EN TRADING - MODELOS ORIGNIALES

In [ ]:
#PRIMER MODELO/first model

In [ ]:
# -*- coding: utf-8 -*-
# Importar las bibliotecas necesarias
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.ensemble import IsolationForest
import yfinance as yf
import ccxt
from datetime import datetime, timedelta
import time

def get_real_data(ticker, period='1y'):
    """
    Descarga datos históricos de trading de Yahoo Finance. Si falla, usa una API de intercambio
    de criptomonedas (ccxt) como respaldo.

    Parámetros:
    ticker (str): Símbolo de la acción o criptomoneda (ej. 'AAPL', 'BTC-USD').
    period (str): Período de tiempo para los datos (ej. '1mo', '1y', '5y').

    Retorna:
    pd.DataFrame: DataFrame con los datos de precios o None si hay un error.
    """
    data = None
    
    # Intento 1: Descargar desde Yahoo Finance con yf.Ticker().history()
    source = 'Yahoo Finance'
    print(f"Intentando descargar datos desde {source} para el ticker: {ticker}...")
    try:
        data = yf.Ticker(ticker).history(period=period, auto_adjust=False)
        
        if data.empty:
            print(f"No se encontraron datos en {source} para el ticker: {ticker}.")
            data = None
        else:
            if isinstance(data.columns, pd.MultiIndex):
                data.columns = data.columns.droplevel()
            
            price_col = None
            for col in ['Close', 'close', 'Adj Close']:
                if col in data.columns:
                    price_col = col
                    break
            
            if not price_col:
                print(f"No se encontró una columna de precios válida en los datos de {source}.")
                data = None
            else:
                data.rename(columns={price_col: 'price'}, inplace=True)
                data = data[['price']]
                print(f"Descarga exitosa desde {source}.")
    except Exception as e:
        print(f"Ocurrió un error con {source}: {e}.")
        data = None

    # Intento 2 (Respaldo): Usar una API de criptomonedas (ccxt)
    if data is None and ticker.endswith('-USD'):
        source = 'ccxt'
        print(f"Intentando descargar datos desde {source} para el ticker: {ticker}...")
        try:
            exchange = ccxt.binance()
            # Intenta con diferentes formatos de ticker
            symbol_candidates = [
                ticker.replace('-', '/'), # ETH-USD -> ETH/USD
                ticker.replace('-USD', '/USDT'), # ETH-USD -> ETH/USDT
                ticker.split('-')[0] + '/USD', # SOL-USD -> SOL/USD
                ticker.split('-')[0] + '/USDT' # SOL-USD -> SOL/USDT
            ]
            
            found_symbol = None
            for symbol in symbol_candidates:
                if symbol in exchange.markets:
                    found_symbol = symbol
                    break
            
            if not found_symbol:
                print(f"El ticker '{ticker}' no se encontró en un formato compatible con {source}.")
            else:
                print(f"Encontrado el símbolo compatible '{found_symbol}'. Descargando datos...")
                since = exchange.parse8601((datetime.now() - timedelta(days=365)).isoformat())
                ohlcv = exchange.fetch_ohlcv(found_symbol, '1d', since)
                
                if not ohlcv:
                    print(f"No se encontraron datos en {source} para el ticker: {found_symbol}.")
                else:
                    data = pd.DataFrame(ohlcv, columns=['timestamp', 'open', 'high', 'low', 'close', 'volume'])
                    data['timestamp'] = pd.to_datetime(data['timestamp'], unit='ms')
                    data.set_index('timestamp', inplace=True)
                    data.rename(columns={'close': 'price'}, inplace=True)
                    data = data[['price']]
                    print(f"Descarga exitosa desde {source}.")
        except Exception as e:
            print(f"Ocurrió un error con {source}: {e}")

    return data

def plot_anomalies(df, predictions):
    """
    Visualiza los datos y resalta las anomalías detectadas.
    
    Parámetros:
    df (pd.DataFrame): DataFrame con los datos de precios.
    predictions (np.array): Array con las predicciones del modelo.
                            -1 para anomalías, 1 para puntos normales.
    """
    df['anomaly_prediction'] = predictions
    anomalies = df[df['anomaly_prediction'] == -1]
    
    plt.figure(figsize=(15, 7))
    plt.title('Detección de Anomalías en Datos de Trading')
    plt.plot(df.index, df['price'], label='Precio (Normal)')
    plt.scatter(anomalies.index, anomalies['price'], color='red', s=50, label='Anomalía Detectada', zorder=2)
    plt.xlabel('Fecha')
    plt.ylabel('Precio')
    plt.legend()
    plt.grid(True, linestyle='--', alpha=0.6)
    plt.show()

def main():
    """
    Función principal para ejecutar el detector de anomalías.
    """
    ticker = input("Introduce el ticker de la acción o criptomoneda (ej. AAPL, BTC-USD) según Yahoo Finance. Si la descarga falla, buscaremos automáticamente en otra fuente: ").upper()
    
    data_df = get_real_data(ticker)
    
    if data_df is not None and not data_df.empty:
        model = IsolationForest(n_estimators=100, contamination=0.01, random_state=42)
        print("Entrenando el modelo Isolation Forest...")
        model.fit(data_df[['price']])
        
        predictions = model.predict(data_df[['price']])
        
        num_anomalies = (predictions == -1).sum()
        print(f"Número de anomalías detectadas: {num_anomalies}")
        
        plot_anomalies(data_df, predictions)
        print("Visualización completada.")
    else:
        print("No se pudo obtener datos válidos para el análisis.")

if __name__ == "__main__":
    main()


In [ ]:
#Segundo Modelo

In [ ]:
# -*- coding: utf-8 -*-
# Importar las bibliotecas necesarias
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.ensemble import IsolationForest
from sklearn.preprocessing import MinMaxScaler
import yfinance as yf
import ccxt
from datetime import datetime, timedelta
import tensorflow as tf
from tensorflow import keras
import ta

def get_real_data(ticker, period='1y'):
    """
    Descarga datos históricos de trading de Yahoo Finance. Si falla, usa una API de intercambio
    de criptomonedas (ccxt) como respaldo.

    Parámetros:
    ticker (str): Símbolo de la acción o criptomoneda (ej. 'AAPL', 'BTC-USD').
    period (str): Período de tiempo para los datos (ej. '1mo', '1y', '5y').

    Retorna:
    pd.DataFrame: DataFrame con los datos de precios y otras características o None si hay un error.
    """
    data = None
    
    # Intento 1: Descargar desde Yahoo Finance con yf.Ticker().history()
    source = 'Yahoo Finance'
    print(f"Intentando descargar datos desde {source} para el ticker: {ticker}...")
    try:
        data = yf.Ticker(ticker).history(period=period, auto_adjust=True)
        
        if data.empty:
            print(f"No se encontraron datos en {source} para el ticker: {ticker}.")
            data = None
        else:
            if isinstance(data.columns, pd.MultiIndex):
                data.columns = data.columns.droplevel()
            
            price_col = None
            for col in ['Close', 'close', 'Adj Close']:
                if col in data.columns:
                    price_col = col
                    break
            
            if not price_col:
                print(f"No se encontró una columna de precios válida en los datos de {source}.")
                data = None
            else:
                # Renombrar todas las columnas relevantes a minúsculas para consistencia
                data.rename(columns={price_col: 'price', 'Open': 'open', 'High': 'high', 'Low': 'low', 'Volume': 'volume'}, inplace=True)
                data = data[['price', 'open', 'high', 'low', 'volume']]
                print(f"Descarga exitosa desde {source}.")
    except Exception as e:
        print(f"Ocurrió un error con {source}: {e}.")
        data = None

    # Intento 2 (Respaldo): Usar una API de criptomonedas (ccxt)
    if data is None and ticker.endswith('-USD'):
        source = 'ccxt'
        print(f"Intentando descargar datos desde {source} para el ticker: {ticker}...")
        try:
            exchange = ccxt.binance()
            # Intenta con diferentes formatos de ticker
            symbol_candidates = [
                ticker.replace('-', '/'), # ETH-USD -> ETH/USD
                ticker.replace('-USD', '/USDT'), # ETH-USD -> ETH/USDT
                ticker.split('-')[0] + '/USD', # SOL-USD -> SOL/USD
                ticker.split('-')[0] + '/USDT' # SOL-USD -> SOL/USDT
            ]
            
            found_symbol = None
            for symbol in symbol_candidates:
                if symbol in exchange.markets:
                    found_symbol = symbol
                    break
            
            if not found_symbol:
                print(f"El ticker '{ticker}' no se encontró en un formato compatible con {source}.")
            else:
                print(f"Encontrado el símbolo compatible '{found_symbol}'. Descargando datos...")
                since = exchange.parse8601((datetime.now() - timedelta(days=365)).isoformat())
                ohlcv = exchange.fetch_ohlcv(found_symbol, '1d', since)
                
                if not ohlcv:
                    print(f"No se encontraron datos en {source} para el ticker: {found_symbol}.")
                else:
                    data = pd.DataFrame(ohlcv, columns=['timestamp', 'open', 'high', 'low', 'close', 'volume'])
                    data['timestamp'] = pd.to_datetime(data['timestamp'], unit='ms')
                    data.set_index('timestamp', inplace=True)
                    data.rename(columns={'close': 'price'}, inplace=True)
                    data = data[['price', 'open', 'high', 'low', 'volume']]
                    print(f"Descarga exitosa desde {source}.")
        except Exception as e:
            print(f"Ocurrió un error con {source}: {e}")

    if data is not None and not data.empty:
        # Añadir indicadores técnicos para un análisis más completo
        # RSI (Relative Strength Index)
        data['RSI'] = ta.momentum.RSIIndicator(data['price']).rsi()
        # SMA (Simple Moving Average)
        data['SMA'] = ta.trend.sma_indicator(data['price'], window=20)
        # ATR (Average True Range)
        data['ATR'] = ta.volatility.AverageTrueRange(data['high'], data['low'], data['price']).average_true_range()
        
        # Eliminar las filas con valores NaN (causadas por los indicadores)
        data.dropna(inplace=True)
        
    return data

def train_autoencoder(X_train):
    """
    Define y entrena un modelo Autoencoder para la detección de anomalías.

    Parámetros:
    X_train (np.array): Datos de entrenamiento.

    Retorna:
    tf.keras.Model: Modelo Autoencoder entrenado.
    """
    input_dim = X_train.shape[1]
    
    # Definir la arquitectura del Autoencoder
    model = keras.Sequential([
        keras.layers.Dense(32, activation='relu', input_shape=(input_dim,)),
        keras.layers.Dense(16, activation='relu'),
        keras.layers.Dense(8, activation='relu'),
        keras.layers.Dense(16, activation='relu'),
        keras.layers.Dense(32, activation='relu'),
        keras.layers.Dense(input_dim, activation='sigmoid')
    ])

    model.compile(optimizer='adam', loss='mean_squared_error')
    
    print("Entrenando el modelo Autoencoder...")
    model.fit(X_train, X_train, 
              epochs=50, 
              batch_size=128, 
              validation_split=0.1,
              verbose=1)
    return model

def plot_anomalies(df, predictions):
    """
    Visualiza los datos y resalta las anomalías detectadas.
    
    Parámetros:
    df (pd.DataFrame): DataFrame con los datos de precios.
    predictions (np.array): Array con las predicciones del modelo.
                            -1 para anomalías, 1 para puntos normales (IsolationForest).
                            Reconstruction error (Autoencoder).
    """
    if 'IsolationForest' in df.columns:
        anomalies = df[df['IsolationForest'] == -1]
        title = 'Detección de Anomalías con Isolation Forest'
        prediction_label = 'Anomalía Detectada'
    else: # Autoencoder
        # Define un umbral para las anomalías. Se puede ajustar.
        threshold = np.mean(predictions) + 2 * np.std(predictions)
        df['anomaly_prediction'] = [1 if error > threshold else 0 for error in predictions]
        anomalies = df[df['anomaly_prediction'] == 1]
        title = 'Detección de Anomalías con Autoencoder'
        prediction_label = 'Anomalía (Reconstruction Error)'

    plt.figure(figsize=(15, 7))
    plt.title(title)
    plt.plot(df.index, df['price'], label='Precio (Normal)')
    plt.scatter(anomalies.index, anomalies['price'], color='red', s=50, label=prediction_label, zorder=2)
    plt.xlabel('Fecha')
    plt.ylabel('Precio')
    plt.legend()
    plt.grid(True, linestyle='--', alpha=0.6)
    plt.show()

def main():
    """
    Función principal para ejecutar el detector de anomalías.
    """
    ticker = input("Introduce el ticker de la acción o criptomoneda (ej. AAPL, BTC-USD) según Yahoo Finance. Si la descarga falla, buscaremos automáticamente en otra fuente: ").upper()
    
    data_df = get_real_data(ticker)
    
    if data_df is not None and not data_df.empty:
        print("\n--- Modelos de Detección de Anomalías ---")
        print("1. Isolation Forest (modelo más simple, rápido)")
        print("2. Autoencoder (modelo de IA más avanzado)")
        model_choice = input("Elige el modelo a usar (1 o 2): ")

        if model_choice == '1':
            model = IsolationForest(n_estimators=100, contamination=0.01, random_state=42)
            print("Entrenando el modelo Isolation Forest...")
            features = data_df.columns.tolist()
            model.fit(data_df[features])
            predictions = model.predict(data_df[features])
            data_df['IsolationForest'] = predictions
            num_anomalies = (predictions == -1).sum()
            print(f"Número de anomalías detectadas: {num_anomalies}")
            plot_anomalies(data_df, predictions)

        elif model_choice == '2':
            # Normalizar los datos para el Autoencoder
            scaler = MinMaxScaler()
            scaled_data = scaler.fit_transform(data_df)
            
            autoencoder = train_autoencoder(scaled_data)
            
            # Obtener los errores de reconstrucción
            reconstructions = autoencoder.predict(scaled_data)
            mse = np.mean(np.power(scaled_data - reconstructions, 2), axis=1)
            
            plot_anomalies(data_df, mse)
        else:
            print("Elección de modelo inválida. Por favor, reinicia el script.")
    else:
        print("No se pudo obtener datos válidos para el análisis.")

if __name__ == "__main__":
    main()

Introduce el ticker de la acción o criptomoneda (ej. AAPL, BTC-USD) según Yahoo Finance. Si la descarga falla, buscaremos automáticamente en otra fuente:  s


Intentando descargar datos desde Yahoo Finance para el ticker: S...
Descarga exitosa desde Yahoo Finance.

--- Modelos de Detección de Anomalías ---
1. Isolation Forest (modelo más simple, rápido)
2. Autoencoder (modelo de IA más avanzado)


Elige el modelo a usar (1 o 2):  2


C:\Users\josep\anaconda3\Lib\site-packages\keras\src\layers\core\dense.py:92: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Entrenando el modelo Autoencoder...
Epoch 1/50


In [ ]:
#Tercer MODELO

In [ ]:
# -*- coding: utf-8 -*-
# Importar las bibliotecas necesarias
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.ensemble import IsolationForest
from sklearn.svm import OneClassSVM
from sklearn.preprocessing import MinMaxScaler
import yfinance as yf
import ccxt
from datetime import datetime, timedelta
import tensorflow as tf
from tensorflow import keras
import ta
import sys

def get_real_data(ticker, period='1y'):
    """
    Descarga datos históricos de trading de Yahoo Finance. Si falla, usa una API de intercambio
    de criptomonedas (ccxt) como respaldo.

    Parámetros:
    ticker (str): Símbolo de la acción o criptomoneda (ej. 'AAPL', 'BTC-USD').
    period (str): Período de tiempo para los datos (ej. '1mo', '1y', '5y').

    Retorna:
    pd.DataFrame: DataFrame con los datos de precios y otras características o None si hay un error.
    """
    data = None
    
    # Intento 1: Descargar desde Yahoo Finance con yf.Ticker().history()
    source = 'Yahoo Finance'
    print(f"Intentando descargar datos desde {source} para el ticker: {ticker}...")
    try:
        data = yf.Ticker(ticker).history(period=period, auto_adjust=True)
        
        if data.empty:
            print(f"No se encontraron datos en {source} para el ticker: {ticker}.")
            data = None
        else:
            if isinstance(data.columns, pd.MultiIndex):
                data.columns = data.columns.droplevel()
            
            price_col = None
            for col in ['Close', 'close', 'Adj Close']:
                if col in data.columns:
                    price_col = col
                    break
            
            if not price_col:
                print(f"No se encontró una columna de precios válida en los datos de {source}.")
                data = None
            else:
                data.rename(columns={price_col: 'price', 'Open': 'open', 'High': 'high', 'Low': 'low', 'Volume': 'volume'}, inplace=True)
                data = data[['price', 'open', 'high', 'low', 'volume']]
                print(f"Descarga exitosa desde {source}.")
    except Exception as e:
        print(f"Ocurrió un error con {source}: {e}.")
        data = None

    # Intento 2 (Respaldo): Usar una API de criptomonedas (ccxt)
    if data is None and ticker.endswith('-USD'):
        source = 'ccxt'
        print(f"Intentando descargar datos desde {source} para el ticker: {ticker}...")
        try:
            exchange = ccxt.binance()
            symbol_candidates = [
                ticker.replace('-', '/'),
                ticker.replace('-USD', '/USDT'),
                ticker.split('-')[0] + '/USD',
                ticker.split('-')[0] + '/USDT'
            ]
            
            found_symbol = None
            for symbol in symbol_candidates:
                if symbol in exchange.markets:
                    found_symbol = symbol
                    break
            
            if not found_symbol:
                print(f"El ticker '{ticker}' no se encontró en un formato compatible con {source}.")
            else:
                print(f"Encontrado el símbolo compatible '{found_symbol}'. Descargando datos...")
                since = exchange.parse8601((datetime.now() - timedelta(days=365)).isoformat())
                ohlcv = exchange.fetch_ohlcv(found_symbol, '1d', since)
                
                if not ohlcv:
                    print(f"No se encontraron datos en {source} para el ticker: {found_symbol}.")
                else:
                    data = pd.DataFrame(ohlcv, columns=['timestamp', 'open', 'high', 'low', 'close', 'volume'])
                    data['timestamp'] = pd.to_datetime(data['timestamp'], unit='ms')
                    data.set_index('timestamp', inplace=True)
                    data.rename(columns={'close': 'price'}, inplace=True)
                    data = data[['price', 'open', 'high', 'low', 'volume']]
                    print(f"Descarga exitosa desde {source}.")
        except Exception as e:
            print(f"Ocurrió un error con {source}: {e}")

    if data is not None and not data.empty:
        # Añadir indicadores técnicos para un análisis más completo
        data['RSI'] = ta.momentum.RSIIndicator(data['price']).rsi()
        data['SMA'] = ta.trend.sma_indicator(data['price'], window=20)
        data['ATR'] = ta.volatility.AverageTrueRange(data['high'], data['low'], data['price']).average_true_range()
        # Nuevos indicadores
        data['MACD'] = ta.trend.macd(data['price'])
        bollinger = ta.volatility.BollingerBands(data['price'])
        data['BBL'] = bollinger.bollinger_lband()
        # CORRECCIÓN: El método para la banda media es 'bollinger_mavg'
        data['BBM'] = bollinger.bollinger_mavg()
        data['BBU'] = bollinger.bollinger_hband()
        
        # Eliminar las filas con valores NaN (causadas por los indicadores)
        data.dropna(inplace=True)
        
    return data

def train_autoencoder(X_train):
    """
    Define y entrena un modelo Autoencoder para la detección de anomalías.

    Parámetros:
    X_train (np.array): Datos de entrenamiento.

    Retorna:
    tf.keras.Model: Modelo Autoencoder entrenado.
    """
    input_dim = X_train.shape[1]
    
    model = keras.Sequential([
        keras.layers.Dense(32, activation='relu', input_shape=(input_dim,)),
        keras.layers.Dense(16, activation='relu'),
        keras.layers.Dense(8, activation='relu'),
        keras.layers.Dense(16, activation='relu'),
        keras.layers.Dense(32, activation='relu'),
        keras.layers.Dense(input_dim, activation='sigmoid')
    ])

    model.compile(optimizer='adam', loss='mean_squared_error')
    
    print("Entrenando el modelo Autoencoder...")
    model.fit(X_train, X_train, 
              epochs=50, 
              batch_size=128, 
              validation_split=0.1,
              verbose=1)
    return model

def create_sequences(data, sequence_length):
    """
    Crea secuencias de datos para el entrenamiento y prueba de un modelo LSTM.
    
    Parámetros:
    data (np.array): Datos normalizados.
    sequence_length (int): Longitud de la secuencia de tiempo.

    Retorna:
    tuple: (X, y) donde X son las secuencias e y son los valores a predecir.
    """
    X, y = [], []
    for i in range(len(data) - sequence_length):
        X.append(data[i:(i + sequence_length)])
        y.append(data[i + sequence_length, 0])
    return np.array(X), np.array(y)

def train_lstm_model(X_train, y_train):
    """
    Define y entrena un modelo LSTM para la predicción de series de tiempo.

    Parámetros:
    X_train (np.array): Secuencias de datos de entrenamiento.
    y_train (np.array): Valores de salida correspondientes.

    Retorna:
    tf.keras.Model: Modelo LSTM entrenado.
    """
    sequence_length = X_train.shape[1]
    n_features = X_train.shape[2]
    
    model = keras.Sequential([
        keras.layers.LSTM(64, activation='relu', input_shape=(sequence_length, n_features), return_sequences=True),
        keras.layers.LSTM(32, activation='relu', return_sequences=False),
        keras.layers.Dense(1)
    ])

    model.compile(optimizer='adam', loss='mean_squared_error')
    
    print("Entrenando el modelo LSTM para predicción de series de tiempo...")
    model.fit(X_train, y_train, 
              epochs=50, 
              batch_size=128, 
              validation_split=0.1,
              verbose=1)
    return model

def plot_anomalies(df, predictions):
    """
    Visualiza los datos y resalta las anomalías detectadas.
    
    Parámetros:
    df (pd.DataFrame): DataFrame con los datos de precios.
    predictions (np.array): Array con las predicciones del modelo.
    """
    title = 'Detección de Anomalías'
    prediction_label = 'Anomalía Detectada'
    
    if 'IsolationForest' in df.columns:
        anomalies = df[df['IsolationForest'] == -1]
        title += ' con Isolation Forest'
    elif 'OneClassSVM' in df.columns:
        anomalies = df[df['OneClassSVM'] == -1]
        title += ' con One-Class SVM'
    else:
        threshold = np.mean(predictions) + 2 * np.std(predictions)
        df['anomaly_prediction'] = [1 if error > threshold else 0 for error in predictions]
        anomalies = df[df['anomaly_prediction'] == 1]
        title += ' con Autoencoder'
        prediction_label = 'Anomalía (Reconstruction Error)'

    plt.figure(figsize=(15, 7))
    plt.title(title)
    plt.plot(df.index, df['price'], label='Precio (Normal)')
    plt.scatter(anomalies.index, anomalies['price'], color='red', s=50, label=prediction_label, zorder=2)
    plt.xlabel('Fecha')
    plt.ylabel('Precio')
    plt.legend()
    plt.grid(True, linestyle='--', alpha=0.6)
    plt.show()

def plot_lstm_anomalies(df, predictions, scaler):
    """
    Visualiza los datos y resalta las anomalías de predicción detectadas por el modelo LSTM.

    Parámetros:
    df (pd.DataFrame): DataFrame con los datos de precios.
    predictions (np.array): Array de predicciones del modelo LSTM.
    scaler (MinMaxScaler): El scaler utilizado para normalizar los datos.
    """
    df_plot = df.iloc[len(df) - len(predictions):].copy()
    
    # La corrección se realiza aquí: el scaler se ajustó al número total de características
    # por lo tanto, necesitamos un arreglo del mismo tamaño para la desnormalización.
    num_features = df.shape[1]
    predictions_denormalized = scaler.inverse_transform(np.c_[predictions, np.zeros((len(predictions), num_features - 1))])[:, 0]
    
    df_plot['predicted_price'] = predictions_denormalized
    df_plot['prediction_error'] = np.abs(df_plot['price'] - df_plot['predicted_price'])
    
    threshold = np.mean(df_plot['prediction_error']) + 2 * np.std(df_plot['prediction_error'])
    df_plot['anomaly'] = df_plot['prediction_error'] > threshold
    
    anomalies = df_plot[df_plot['anomaly']]

    plt.figure(figsize=(15, 7))
    plt.title('Detección de Anomalías Futuras con LSTM')
    plt.plot(df_plot.index, df_plot['price'], label='Precio Real', color='blue')
    plt.plot(df_plot.index, df_plot['predicted_price'], label='Precio Predicho (LSTM)', color='green', linestyle='--')
    plt.scatter(anomalies.index, anomalies['price'], color='red', s=50, label='Anomalía de Predicción', zorder=2)
    plt.xlabel('Fecha')
    plt.ylabel('Precio')
    plt.legend()
    plt.grid(True, linestyle='--', alpha=0.6)
    plt.show()

def generate_recommendation(anomalies, model_type):
    """
    Genera una recomendación de trading basada en las anomalías detectadas.

    Parámetros:
    anomalies (pd.DataFrame): DataFrame con los puntos anómalos.
    model_type (str): El modelo utilizado ('IsolationForest', 'Autoencoder', 'LSTM').

    Retorna:
    tuple: (str, str) con el resumen y la recomendación.
    """
    if anomalies.empty:
        summary = "El análisis no ha detectado anomalías significativas en los datos recientes."
        recommendation = "MANTENER"
        reason = "El mercado parece estable según el análisis del modelo. No se han identificado movimientos de precio inusuales en los últimos días."
        return summary, recommendation, reason

    latest_anomaly = anomalies.iloc[-1]
    summary = f"El modelo ha detectado una anomalía reciente el {latest_anomaly.name.strftime('%Y-%m-%d')}."
    
    last_price = latest_anomaly['price']
    
    # Lógica mejorada que considera la dirección de la anomalía y el volumen
    if latest_anomaly.name == anomalies.index[-1]:
        price_change = last_price - latest_anomaly['open']
        if price_change > 0:
            if latest_anomaly['volume'] > anomalies['volume'].mean():
                recommendation = "VENDER"
                reason = "Se ha detectado un pico de precio significativo con un volumen alto, lo que podría indicar una corrección inminente."
            else:
                recommendation = "MANTENER"
                reason = "El precio ha subido, pero el volumen es bajo, lo que podría ser una señal débil de compra. Se recomienda esperar."
        else:
            if latest_anomaly['volume'] > anomalies['volume'].mean():
                recommendation = "COMPRAR"
                reason = "Se ha detectado una caída de precio abrupta con un volumen alto. Esto podría ser una oportunidad de compra atractiva."
            else:
                recommendation = "MANTENER"
                reason = "El precio ha bajado, pero el volumen es bajo. La señal no es lo suficientemente fuerte para sugerir una compra inmediata."
    else:
        # Recomendación genérica si la última anomalía no es la más reciente
        recommendation = "MANTENER"
        reason = "Se ha detectado una anomalía en el pasado, pero no se ha identificado un evento reciente que requiera una acción inmediata."
    
    return summary, recommendation, reason

def run_backtest(df, anomalies):
    """
    Simula una estrategia de trading simple basada en las anomalías.
    Estrategia: Comprar al detectar una anomalía de precio bajo y vender al detectar una
    anomalía de precio alto, o al final de los datos.

    Parámetros:
    df (pd.DataFrame): DataFrame con todos los datos.
    anomalies (pd.DataFrame): DataFrame con las anomalías detectadas.

    Retorna:
    pd.DataFrame: DataFrame con los resultados del backtest.
    """
    if anomalies.empty:
        print("No se pueden ejecutar el backtest, no hay anomalías detectadas.")
        return pd.DataFrame()
        
    portfolio = pd.DataFrame(index=df.index)
    portfolio['asset_price'] = df['price']
    portfolio['holdings'] = 0
    portfolio['cash'] = 1000  # Capital inicial
    
    buy_signals = anomalies[anomalies['price'] < df['price'].mean()]
    sell_signals = anomalies[anomalies['price'] > df['price'].mean()]

    in_position = False
    
    for i in range(len(portfolio)):
        if i > 0:
            portfolio.loc[portfolio.index[i], 'cash'] = portfolio.loc[portfolio.index[i-1], 'cash']
            portfolio.loc[portfolio.index[i], 'holdings'] = portfolio.loc[portfolio.index[i-1], 'holdings']
        
        current_date = portfolio.index[i]
        
        # Comprar
        if not in_position and current_date in buy_signals.index:
            shares_to_buy = portfolio.loc[current_date, 'cash'] / portfolio.loc[current_date, 'asset_price']
            portfolio.loc[current_date, 'holdings'] = shares_to_buy
            portfolio.loc[current_date, 'cash'] = 0
            in_position = True
            print(f"COMPRA: el {current_date.strftime('%Y-%m-%d')}, precio: ${portfolio.loc[current_date, 'asset_price']:.2f}")

        # Vender
        elif in_position and current_date in sell_signals.index:
            cash_from_sale = portfolio.loc[current_date, 'holdings'] * portfolio.loc[current_date, 'asset_price']
            portfolio.loc[current_date, 'cash'] = cash_from_sale
            portfolio.loc[current_date, 'holdings'] = 0
            in_position = False
            print(f"VENTA: el {current_date.strftime('%Y-%m-%d')}, precio: ${portfolio.loc[current_date, 'asset_price']:.2f}")

    portfolio['total_value'] = portfolio['cash'] + portfolio['holdings'] * portfolio['asset_price']
    
    initial_value = portfolio['total_value'].iloc[0]
    final_value = portfolio['total_value'].iloc[-1]
    
    print("\n--- Resultados del Backtest ---")
    print(f"Valor inicial del portafolio: ${initial_value:.2f}")
    print(f"Valor final del portafolio: ${final_value:.2f}")
    
    # Calcular el retorno del backtest y del mercado
    backtest_return = (final_value - initial_value) / initial_value
    market_return = (df['price'].iloc[-1] - df['price'].iloc[0]) / df['price'].iloc[0]
    
    print(f"Retorno del Backtest: {backtest_return * 100:.2f}%")
    print(f"Retorno del mercado: {market_return * 100:.2f}%")
    
    plt.figure(figsize=(15, 7))
    plt.title('Rendimiento del Backtest vs. Mercado')
    plt.plot(portfolio.index, portfolio['total_value'], label='Valor del Portafolio (Estrategia de Anomalías)', color='purple')
    plt.plot(df.index, (df['price'] / df['price'].iloc[0]) * initial_value, label='Rendimiento del Mercado', color='orange', linestyle='--')
    plt.xlabel('Fecha')
    plt.ylabel('Valor del Portafolio ($)')
    plt.legend()
    plt.grid(True, linestyle='--', alpha=0.6)
    plt.show()

    return portfolio

def export_to_excel(df, anomalies, backtest_results, ticker, model_type):
    """
    Exporta el DataFrame completo, un resumen de las anomalías y los resultados del backtest
    a un archivo de Excel.

    Parámetros:
    df (pd.DataFrame): DataFrame con todos los datos.
    anomalies (pd.DataFrame): DataFrame con las anomalías detectadas.
    backtest_results (pd.DataFrame): DataFrame con los resultados del backtest.
    ticker (str): Ticker del activo.
    model_type (str): El modelo utilizado.
    """
    try:
        filename = f"Analisis_Anomalias_{ticker}_{model_type}.xlsx"
        writer = pd.ExcelWriter(filename, engine='xlsxwriter')
        
        df.to_excel(writer, sheet_name='Datos Completos')
        
        if not anomalies.empty:
            anomalies.to_excel(writer, sheet_name='Anomalias Detectadas')
        else:
            empty_df = pd.DataFrame(["No se detectaron anomalías."], columns=["Resultado"])
            empty_df.to_excel(writer, sheet_name='Anomalias Detectadas')
        
        if not backtest_results.empty:
            backtest_results.to_excel(writer, sheet_name='Resultados Backtest')
        
        writer._save()
        print(f"\n¡Análisis exportado a '{filename}' con éxito!")
    except Exception as e:
        print(f"Error al exportar a Excel: {e}")

def main():
    """
    Función principal para ejecutar el detector de anomalías.
    """
    ticker = input("Introduce el ticker de la acción o criptomoneda (ej. AAPL, BTC-USD) según Yahoo Finance. Si la descarga falla, buscaremos automáticamente en otra fuente: ").upper()
    
    data_df = get_real_data(ticker)
    
    if data_df is not None and not data_df.empty:
        print("\n--- Modelos de Detección de Anomalías ---")
        print("1. Isolation Forest (modelo más simple, rápido)")
        print("2. Autoencoder (modelo de IA más avanzado)")
        print("3. LSTM (para predicción y anomalías futuras)")
        print("4. One-Class SVM (para patrones complejos)")
        model_choice = input("Elige el modelo a usar (1, 2, 3 o 4): ")
        
        anomalies = pd.DataFrame() # DataFrame vacío por defecto
        
        if model_choice == '1':
            model_type = 'IsolationForest'
            model = IsolationForest(n_estimators=100, contamination=0.01, random_state=42)
            print("Entrenando el modelo Isolation Forest...")
            features = data_df.columns.tolist()
            model.fit(data_df[features])
            predictions = model.predict(data_df[features])
            data_df['IsolationForest'] = predictions
            anomalies = data_df[data_df['IsolationForest'] == -1]
            print(f"Número de anomalías detectadas: {(predictions == -1).sum()}")
            plot_anomalies(data_df, predictions)

        elif model_choice == '2':
            model_type = 'Autoencoder'
            scaler = MinMaxScaler()
            scaled_data = scaler.fit_transform(data_df)
            
            autoencoder = train_autoencoder(scaled_data)
            
            reconstructions = autoencoder.predict(scaled_data)
            mse = np.mean(np.power(scaled_data - reconstructions, 2), axis=1)
            
            threshold = np.mean(mse) + 2 * np.std(mse)
            data_df['anomaly'] = [True if error > threshold else False for error in mse]
            anomalies = data_df[data_df['anomaly']]
            print(f"Número de anomalías detectadas: {anomalies.shape[0]}")
            plot_anomalies(data_df, mse)

        elif model_choice == '3':
            model_type = 'LSTM'
            sequence_length = 30
            scaler = MinMaxScaler()
            scaled_data = scaler.fit_transform(data_df)
            X, y = create_sequences(scaled_data, sequence_length)
            
            test_size = int(len(X) * 0.2)
            X_train, y_train = X[:-test_size], y[:-test_size]
            X_test, y_test = X[-test_size:], y[-test_size:]
            
            lstm_model = train_lstm_model(X_train, y_train)
            
            predictions = lstm_model.predict(X_test)
            
            data_df_plot = data_df.iloc[len(data_df) - len(predictions):].copy()
            
            num_features = data_df.shape[1]
            predictions_denormalized = scaler.inverse_transform(np.c_[predictions, np.zeros((len(predictions), num_features - 1))])[:, 0]
            data_df_plot['predicted_price'] = predictions_denormalized
            data_df_plot['prediction_error'] = np.abs(data_df_plot['price'] - data_df_plot['predicted_price'])
            
            threshold = np.mean(data_df_plot['prediction_error']) + 2 * np.std(data_df_plot['prediction_error'])
            data_df_plot['anomaly'] = data_df_plot['prediction_error'] > threshold
            anomalies = data_df_plot[data_df_plot['anomaly']]
            print(f"Número de anomalías de predicción detectadas: {anomalies.shape[0]}")
            plot_lstm_anomalies(data_df, predictions, scaler)

        elif model_choice == '4':
            model_type = 'OneClassSVM'
            model = OneClassSVM(kernel='rbf', gamma='auto', nu=0.01)
            scaler = MinMaxScaler()
            scaled_data = scaler.fit_transform(data_df)
            print("Entrenando el modelo One-Class SVM...")
            predictions = model.fit_predict(scaled_data)
            data_df['OneClassSVM'] = predictions
            anomalies = data_df[data_df['OneClassSVM'] == -1]
            print(f"Número de anomalías detectadas: {(predictions == -1).sum()}")
            plot_anomalies(data_df, predictions)

        else:
            print("Elección de modelo inválida. Por favor, reinicia el script.")
            sys.exit()

        print("\n--- Resumen del Análisis ---")
        summary, recommendation, reason = generate_recommendation(anomalies, model_type)
        print(f"Resumen: {summary}")
        print(f"Recomendación: {recommendation}")
        print(f"Motivo: {reason}")
        print("\n*Este análisis es solo para fines informativos y no debe ser considerado un consejo financiero.*")
        
        backtest_results = pd.DataFrame()
        if not anomalies.empty:
            backtest_choice = input("\n¿Deseas realizar un backtest de la estrategia? (s/n): ").lower()
            if backtest_choice == 's':
                backtest_results = run_backtest(data_df, anomalies)

        export_choice = input("\n¿Deseas exportar los resultados a un archivo de Excel? (s/n): ").lower()
        if export_choice == 's':
            export_to_excel(data_df, anomalies, backtest_results, ticker, model_type)
        else:
            print("Análisis terminado. No se ha exportado ningún archivo.")

    else:
        print("No se pudo obtener datos válidos para el análisis.")

if __name__ == "__main__":
    main()


In [ ]:
#CUARTO MODELO

In [ ]:
# -*- coding: utf-8 -*-
# Importar las bibliotecas necesarias
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.ensemble import IsolationForest
from sklearn.svm import OneClassSVM
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import GridSearchCV
import yfinance as yf
import ccxt
from datetime import datetime, timedelta
import tensorflow as tf
from tensorflow import keras
import ta
import sys

def get_real_data(ticker, period='1y'):
    """
    Descarga datos históricos de trading de Yahoo Finance. Si falla, usa una API de intercambio
    de criptomonedas (ccxt) como respaldo.

    Parámetros:
    ticker (str): Símbolo de la acción o criptomoneda (ej. 'AAPL', 'BTC-USD').
    period (str): Período de tiempo para los datos (ej. '1mo', '1y', '5y').

    Retorna:
    pd.DataFrame: DataFrame con los datos de precios y otras características o None si hay un error.
    """
    data = None
    
    # Intento 1: Descargar desde Yahoo Finance con yf.Ticker().history()
    source = 'Yahoo Finance'
    print(f"Intentando descargar datos desde {source} para el ticker: {ticker}...")
    try:
        data = yf.Ticker(ticker).history(period=period, auto_adjust=True)
        
        if data.empty:
            print(f"No se encontraron datos en {source} para el ticker: {ticker}.")
            data = None
        else:
            if isinstance(data.columns, pd.MultiIndex):
                data.columns = data.columns.droplevel()
            
            price_col = None
            for col in ['Close', 'close', 'Adj Close']:
                if col in data.columns:
                    price_col = col
                    break
            
            if not price_col:
                print(f"No se encontró una columna de precios válida en los datos de {source}.")
                data = None
            else:
                data.rename(columns={price_col: 'price', 'Open': 'open', 'High': 'high', 'Low': 'low', 'Volume': 'volume'}, inplace=True)
                data = data[['price', 'open', 'high', 'low', 'volume']]
                print(f"Descarga exitosa desde {source}.")
    except Exception as e:
        print(f"Ocurrió un error con {source}: {e}.")
        data = None

    # Intento 2 (Respaldo): Usar una API de criptomonedas (ccxt)
    if data is None and ticker.endswith('-USD'):
        source = 'ccxt'
        print(f"Intentando descargar datos desde {source} para el ticker: {ticker}...")
        try:
            exchange = ccxt.binance()
            symbol_candidates = [
                ticker.replace('-', '/'),
                ticker.replace('-USD', '/USDT'),
                ticker.split('-')[0] + '/USD',
                ticker.split('-')[0] + '/USDT'
            ]
            
            found_symbol = None
            for symbol in symbol_candidates:
                if symbol in exchange.markets:
                    found_symbol = symbol
                    break
            
            if not found_symbol:
                print(f"El ticker '{ticker}' no se encontró en un formato compatible con {source}.")
            else:
                print(f"Encontrado el símbolo compatible '{found_symbol}'. Descargando datos...")
                since = exchange.parse8601((datetime.now() - timedelta(days=365)).isoformat())
                ohlcv = exchange.fetch_ohlcv(found_symbol, '1d', since)
                
                if not ohlcv:
                    print(f"No se encontraron datos en {source} para el ticker: {found_symbol}.")
                else:
                    data = pd.DataFrame(ohlcv, columns=['timestamp', 'open', 'high', 'low', 'close', 'volume'])
                    data['timestamp'] = pd.to_datetime(data['timestamp'], unit='ms')
                    data.set_index('timestamp', inplace=True)
                    data.rename(columns={'close': 'price'}, inplace=True)
                    data = data[['price', 'open', 'high', 'low', 'volume']]
                    print(f"Descarga exitosa desde {source}.")
        except Exception as e:
            print(f"Ocurrió un error con {source}: {e}")

    if data is not None and not data.empty:
        # Añadir indicadores técnicos para un análisis más completo
        data['RSI'] = ta.momentum.RSIIndicator(data['price']).rsi()
        data['SMA'] = ta.trend.sma_indicator(data['price'], window=20)
        data['ATR'] = ta.volatility.AverageTrueRange(data['high'], data['low'], data['price']).average_true_range()
        # Nuevos indicadores
        data['MACD'] = ta.trend.macd(data['price'])
        data['MACD_Signal'] = ta.trend.macd_signal(data['price'])
        bollinger = ta.volatility.BollingerBands(data['price'])
        data['BBL'] = bollinger.bollinger_lband()
        data['BBM'] = bollinger.bollinger_mavg()
        data['BBU'] = bollinger.bollinger_hband()
        
        # Eliminar las filas con valores NaN (causadas por los indicadores)
        data.dropna(inplace=True)
        
    return data

def optimize_hyperparameters(model_type, data_df):
    """
    Optimiza los hiperparámetros para Isolation Forest o One-Class SVM.

    Parámetros:
    model_type (str): El modelo a optimizar ('IsolationForest' o 'OneClassSVM').
    data_df (pd.DataFrame): DataFrame con los datos de entrada.

    Retorna:
    Un modelo con los mejores parámetros encontrados.
    """
    if model_type == 'IsolationForest':
        model = IsolationForest(random_state=42)
        param_grid = {
            'n_estimators': [50, 100, 200],
            'contamination': [0.01, 0.05, 0.1]
        }
        print("\nOptimizando hiperparámetros para Isolation Forest...")
        grid_search = GridSearchCV(model, param_grid, cv=5, scoring='f1_weighted', verbose=1)
        grid_search.fit(data_df)
        print("Mejores parámetros encontrados: ", grid_search.best_params_)
        return grid_search.best_estimator_
        
    elif model_type == 'OneClassSVM':
        model = OneClassSVM()
        param_grid = {
            'kernel': ['rbf', 'poly', 'sigmoid'],
            'nu': [0.01, 0.05, 0.1],
            'gamma': ['scale', 'auto']
        }
        print("\nOptimizando hiperparámetros para One-Class SVM...")
        grid_search = GridSearchCV(model, param_grid, cv=5, scoring='f1_weighted', verbose=1)
        scaler = MinMaxScaler()
        scaled_data = scaler.fit_transform(data_df)
        grid_search.fit(scaled_data)
        print("Mejores parámetros encontrados: ", grid_search.best_params_)
        return grid_search.best_estimator_
    else:
        return None

def train_autoencoder(X_train):
    """
    Define y entrena un modelo Autoencoder para la detección de anomalías.

    Parámetros:
    X_train (np.array): Datos de entrenamiento.

    Retorna:
    tf.keras.Model: Modelo Autoencoder entrenado.
    """
    input_dim = X_train.shape[1]
    
    model = keras.Sequential([
        keras.layers.Dense(32, activation='relu', input_shape=(input_dim,)),
        keras.layers.Dense(16, activation='relu'),
        keras.layers.Dense(8, activation='relu'),
        keras.layers.Dense(16, activation='relu'),
        keras.layers.Dense(32, activation='relu'),
        keras.layers.Dense(input_dim, activation='sigmoid')
    ])

    model.compile(optimizer='adam', loss='mean_squared_error')
    
    print("Entrenando el modelo Autoencoder...")
    model.fit(X_train, X_train, 
              epochs=50, 
              batch_size=128, 
              validation_split=0.1,
              verbose=1)
    return model

def create_sequences(data, sequence_length):
    """
    Crea secuencias de datos para el entrenamiento y prueba de un modelo LSTM.
    
    Parámetros:
    data (np.array): Datos normalizados.
    sequence_length (int): Longitud de la secuencia de tiempo.

    Retorna:
    tuple: (X, y) donde X son las secuencias e y son los valores a predecir.
    """
    X, y = [], []
    for i in range(len(data) - sequence_length):
        X.append(data[i:(i + sequence_length)])
        y.append(data[i + sequence_length, 0])
    return np.array(X), np.array(y)

def train_lstm_model(X_train, y_train):
    """
    Define y entrena un modelo LSTM para la predicción de series de tiempo.

    Parámetros:
    X_train (np.array): Secuencias de datos de entrenamiento.
    y_train (np.array): Valores de salida correspondientes.

    Retorna:
    tf.keras.Model: Modelo LSTM entrenado.
    """
    sequence_length = X_train.shape[1]
    n_features = X_train.shape[2]
    
    model = keras.Sequential([
        keras.layers.LSTM(64, activation='relu', input_shape=(sequence_length, n_features), return_sequences=True),
        keras.layers.LSTM(32, activation='relu', return_sequences=False),
        keras.layers.Dense(1)
    ])

    model.compile(optimizer='adam', loss='mean_squared_error')
    
    print("Entrenando el modelo LSTM para predicción de series de tiempo...")
    model.fit(X_train, y_train, 
              epochs=50, 
              batch_size=128, 
              validation_split=0.1,
              verbose=1)
    return model

def plot_anomalies(df, predictions, model_type):
    """
    Visualiza los datos y resalta las anomalías detectadas.
    
    Parámetros:
    df (pd.DataFrame): DataFrame con los datos de precios.
    predictions (np.array): Array con las predicciones del modelo.
    model_type (str): El tipo de modelo utilizado.
    """
    title = f'Detección de Anomalías con {model_type}'
    prediction_label = 'Anomalía Detectada'
    
    if model_type in ['IsolationForest', 'OneClassSVM']:
        anomalies = df[df[model_type] == -1]
    else: # Autoencoder
        threshold = np.mean(predictions) + 2 * np.std(predictions)
        df['anomaly_prediction'] = [1 if error > threshold else 0 for error in predictions]
        anomalies = df[df['anomaly_prediction'] == 1]
        prediction_label = 'Anomalía (Reconstruction Error)'

    plt.figure(figsize=(15, 7))
    plt.title(title)
    plt.plot(df.index, df['price'], label='Precio (Normal)')
    plt.scatter(anomalies.index, anomalies['price'], color='red', s=50, label=prediction_label, zorder=2)
    plt.xlabel('Fecha')
    plt.ylabel('Precio')
    plt.legend()
    plt.grid(True, linestyle='--', alpha=0.6)
    plt.show()

def plot_lstm_anomalies(df, predictions, scaler):
    """
    Visualiza los datos y resalta las anomalías de predicción detectadas por el modelo LSTM.

    Parámetros:
    df (pd.DataFrame): DataFrame con los datos de precios.
    predictions (np.array): Array de predicciones del modelo LSTM.
    scaler (MinMaxScaler): El scaler utilizado para normalizar los datos.
    """
    df_plot = df.iloc[len(df) - len(predictions):].copy()
    
    num_features = df.shape[1]
    predictions_denormalized = scaler.inverse_transform(np.c_[predictions, np.zeros((len(predictions), num_features - 1))])[:, 0]
    
    df_plot['predicted_price'] = predictions_denormalized
    df_plot['prediction_error'] = np.abs(df_plot['price'] - df_plot['predicted_price'])
    
    threshold = np.mean(df_plot['prediction_error']) + 2 * np.std(df_plot['prediction_error'])
    df_plot['anomaly'] = df_plot['prediction_error'] > threshold
    
    anomalies = df_plot[df_plot['anomaly']]

    plt.figure(figsize=(15, 7))
    plt.title('Detección de Anomalías Futuras con LSTM')
    plt.plot(df_plot.index, df_plot['price'], label='Precio Real', color='blue')
    plt.plot(df_plot.index, df_plot['predicted_price'], label='Precio Predicho (LSTM)', color='green', linestyle='--')
    plt.scatter(anomalies.index, anomalies['price'], color='red', s=50, label='Anomalía de Predicción', zorder=2)
    plt.xlabel('Fecha')
    plt.ylabel('Precio')
    plt.legend()
    plt.grid(True, linestyle='--', alpha=0.6)
    plt.show()

def generate_recommendation(anomalies, data_df):
    """
    Genera una recomendación de trading basada en la última anomalía detectada
    y los indicadores del último día.

    Parámetros:
    anomalies (pd.DataFrame): DataFrame con los puntos anómalos.
    data_df (pd.DataFrame): DataFrame con todos los datos.

    Retorna:
    tuple: (str, str) con el resumen y la recomendación.
    """
    if anomalies.empty:
        summary = "El análisis no ha detectado anomalías significativas en los datos recientes."
        recommendation = "MANTENER"
        reason = "El mercado parece estable. No se han identificado movimientos de precio inusuales."
        return summary, recommendation, reason

    latest_anomaly = anomalies.iloc[-1]
    last_data = data_df.iloc[-1]

    # Lógica mejorada que considera la dirección de la anomalía y el volumen
    summary = f"El modelo ha detectado una anomalía reciente el {latest_anomaly.name.strftime('%Y-%m-%d')}."
    
    # Evaluar la señal de trading más reciente
    if last_data['MACD'] > last_data['MACD_Signal'] and last_data['MACD'] > 0:
        recommendation = "COMPRAR"
        reason = "La línea MACD ha cruzado por encima de la línea de señal y está en territorio positivo, lo que indica un fuerte impulso alcista."
    elif last_data['MACD'] < last_data['MACD_Signal'] and last_data['MACD'] < 0:
        recommendation = "VENDER"
        reason = "La línea MACD ha cruzado por debajo de la línea de señal y está en territorio negativo, lo que indica un fuerte impulso bajista."
    else:
        recommendation = "MANTENER"
        reason = "Los indicadores no muestran una señal clara. El mercado parece estar en consolidación."

    return summary, recommendation, reason

def run_backtest(df):
    """
    Simula una estrategia de trading basada en el cruce de las líneas MACD y MACD_Signal.
    
    Parámetros:
    df (pd.DataFrame): DataFrame con los datos y los indicadores MACD.

    Retorna:
    pd.DataFrame: DataFrame con los resultados del backtest.
    """
    print("\n--- Ejecutando Backtest con Estrategia de Cruce MACD ---")
    portfolio = pd.DataFrame(index=df.index)
    portfolio['asset_price'] = df['price']
    portfolio['holdings'] = 0.0
    portfolio['cash'] = 1000.0  # Capital inicial
    
    in_position = False
    
    for i in range(1, len(portfolio)):
        portfolio.loc[portfolio.index[i], 'cash'] = portfolio.loc[portfolio.index[i-1], 'cash']
        portfolio.loc[portfolio.index[i], 'holdings'] = portfolio.loc[portfolio.index[i-1], 'holdings']
        
        # Señal de compra: MACD cruza por encima de la línea de señal
        if df['MACD'].iloc[i] > df['MACD_Signal'].iloc[i] and df['MACD'].iloc[i-1] <= df['MACD_Signal'].iloc[i-1] and not in_position:
            if portfolio.loc[portfolio.index[i], 'cash'] > 0:
                shares_to_buy = portfolio.loc[portfolio.index[i], 'cash'] / portfolio.loc[portfolio.index[i], 'asset_price']
                portfolio.loc[portfolio.index[i], 'holdings'] += shares_to_buy
                portfolio.loc[portfolio.index[i], 'cash'] = 0.0
                in_position = True
                print(f"COMPRA: el {df.index[i].strftime('%Y-%m-%d')}, precio: ${df['price'].iloc[i]:.2f}")
        
        # Señal de venta: MACD cruza por debajo de la línea de señal
        elif df['MACD'].iloc[i] < df['MACD_Signal'].iloc[i] and df['MACD'].iloc[i-1] >= df['MACD_Signal'].iloc[i-1] and in_position:
            cash_from_sale = portfolio.loc[portfolio.index[i], 'holdings'] * portfolio.loc[portfolio.index[i], 'asset_price']
            portfolio.loc[portfolio.index[i], 'cash'] = cash_from_sale
            portfolio.loc[portfolio.index[i], 'holdings'] = 0.0
            in_position = False
            print(f"VENTA: el {df.index[i].strftime('%Y-%m-%d')}, precio: ${df['price'].iloc[i]:.2f}")

    portfolio['total_value'] = portfolio['cash'] + portfolio['holdings'] * portfolio['asset_price']
    
    initial_value = portfolio['total_value'].iloc[0]
    final_value = portfolio['total_value'].iloc[-1]
    
    print("\n--- Resultados del Backtest ---")
    print(f"Valor inicial del portafolio: ${initial_value:.2f}")
    print(f"Valor final del portafolio: ${final_value:.2f}")
    
    backtest_return = (final_value - initial_value) / initial_value
    market_return = (df['price'].iloc[-1] - df['price'].iloc[0]) / df['price'].iloc[0]
    
    print(f"Retorno del Backtest: {backtest_return * 100:.2f}%")
    print(f"Retorno del mercado: {market_return * 100:.2f}%")
    
    plt.figure(figsize=(15, 7))
    plt.title('Rendimiento del Backtest vs. Mercado')
    plt.plot(portfolio.index, portfolio['total_value'], label='Valor del Portafolio (Estrategia de Cruce MACD)', color='purple')
    plt.plot(df.index, (df['price'] / df['price'].iloc[0]) * initial_value, label='Rendimiento del Mercado', color='orange', linestyle='--')
    plt.xlabel('Fecha')
    plt.ylabel('Valor del Portafolio ($)')
    plt.legend()
    plt.grid(True, linestyle='--', alpha=0.6)
    plt.show()

    return portfolio

def export_to_excel(df, anomalies, backtest_results, ticker, model_type):
    """
    Exporta el DataFrame completo, un resumen de las anomalías y los resultados del backtest
    a un archivo de Excel.

    Parámetros:
    df (pd.DataFrame): DataFrame con todos los datos.
    anomalies (pd.DataFrame): DataFrame con las anomalías detectadas.
    backtest_results (pd.DataFrame): DataFrame con los resultados del backtest.
    ticker (str): Ticker del activo.
    model_type (str): El modelo utilizado.
    """
    try:
        filename = f"Analisis_Anomalias_{ticker}_{model_type}.xlsx"
        writer = pd.ExcelWriter(filename, engine='xlsxwriter')
        
        df.to_excel(writer, sheet_name='Datos Completos')
        
        if not anomalies.empty:
            anomalies.to_excel(writer, sheet_name='Anomalias Detectadas')
        else:
            empty_df = pd.DataFrame(["No se detectaron anomalías."], columns=["Resultado"])
            empty_df.to_excel(writer, sheet_name='Anomalias Detectadas')
        
        if not backtest_results.empty:
            backtest_results.to_excel(writer, sheet_name='Resultados Backtest')
        
        writer._save()
        print(f"\n¡Análisis exportado a '{filename}' con éxito!")
    except Exception as e:
        print(f"Error al exportar a Excel: {e}")

def main():
    """
    Función principal para ejecutar el detector de anomalías.
    """
    ticker = input("Introduce el ticker de la acción o criptomoneda (ej. AAPL, BTC-USD) según Yahoo Finance. Si la descarga falla, buscaremos automáticamente en otra fuente: ").upper()
    
    data_df = get_real_data(ticker)
    
    if data_df is not None and not data_df.empty:
        print("\n--- Modelos de Detección de Anomalías ---")
        print("1. Isolation Forest (modelo más simple y rápido)")
        print("2. Autoencoder (modelo de IA avanzado)")
        print("3. LSTM (para predicción y anomalías futuras)")
        print("4. One-Class SVM (para patrones complejos)")
        model_choice = input("Elige el modelo a usar (1, 2, 3 o 4): ")
        
        anomalies = pd.DataFrame() # DataFrame vacío por defecto
        
        if model_choice == '1':
            model_type = 'IsolationForest'
            model = optimize_hyperparameters(model_type, data_df)
            predictions = model.fit_predict(data_df)
            data_df[model_type] = predictions
            anomalies = data_df[data_df[model_type] == -1]
            print(f"Número de anomalías detectadas: {(predictions == -1).sum()}")
            plot_anomalies(data_df, predictions, model_type)
        
        elif model_choice == '2':
            model_type = 'Autoencoder'
            scaler = MinMaxScaler()
            scaled_data = scaler.fit_transform(data_df)
            
            autoencoder = train_autoencoder(scaled_data)
            
            reconstructions = autoencoder.predict(scaled_data)
            mse = np.mean(np.power(scaled_data - reconstructions, 2), axis=1)
            
            threshold = np.mean(mse) + 2 * np.std(mse)
            data_df['anomaly'] = [True if error > threshold else False for error in mse]
            anomalies = data_df[data_df['anomaly']]
            print(f"Número de anomalías detectadas: {anomalies.shape[0]}")
            plot_anomalies(data_df, mse, model_type)

        elif model_choice == '3':
            model_type = 'LSTM'
            sequence_length = 30
            scaler = MinMaxScaler()
            scaled_data = scaler.fit_transform(data_df)
            X, y = create_sequences(scaled_data, sequence_length)
            
            test_size = int(len(X) * 0.2)
            X_train, y_train = X[:-test_size], y[:-test_size]
            X_test, y_test = X[-test_size:], y[-test_size:]
            
            lstm_model = train_lstm_model(X_train, y_train)
            
            predictions = lstm_model.predict(X_test)
            
            data_df_plot = data_df.iloc[len(data_df) - len(predictions):].copy()
            
            num_features = data_df.shape[1]
            predictions_denormalized = scaler.inverse_transform(np.c_[predictions, np.zeros((len(predictions), num_features - 1))])[:, 0]
            data_df_plot['predicted_price'] = predictions_denormalized
            data_df_plot['prediction_error'] = np.abs(data_df_plot['price'] - data_df_plot['predicted_price'])
            
            threshold = np.mean(data_df_plot['prediction_error']) + 2 * np.std(data_df_plot['prediction_error'])
            data_df_plot['anomaly'] = data_df_plot['prediction_error'] > threshold
            anomalies = data_df_plot[data_df_plot['anomaly']]
            print(f"Número de anomalías de predicción detectadas: {anomalies.shape[0]}")
            plot_lstm_anomalies(data_df, predictions, scaler)

        elif model_choice == '4':
            model_type = 'OneClassSVM'
            model = optimize_hyperparameters(model_type, data_df)
            scaler = MinMaxScaler()
            scaled_data = scaler.fit_transform(data_df)
            predictions = model.fit_predict(scaled_data)
            data_df[model_type] = predictions
            anomalies = data_df[data_df[model_type] == -1]
            print(f"Número de anomalías detectadas: {(predictions == -1).sum()}")
            plot_anomalies(data_df, predictions, model_type)

        else:
            print("Elección de modelo inválida. Por favor, reinicia el script.")
            sys.exit()

        print("\n--- Resumen del Análisis ---")
        summary, recommendation, reason = generate_recommendation(anomalies, data_df)
        print(f"Resumen: {summary}")
        print(f"Recomendación: {recommendation}")
        print(f"Motivo: {reason}")
        print("\n*Este análisis es solo para fines informativos y no debe ser considerado un consejo financiero.*")
        
        backtest_results = pd.DataFrame()
        backtest_choice = input("\n¿Deseas realizar un backtest de la estrategia de cruce MACD? (s/n): ").lower()
        if backtest_choice == 's':
            backtest_results = run_backtest(data_df)

        export_choice = input("\n¿Deseas exportar los resultados a un archivo de Excel? (s/n): ").lower()
        if export_choice == 's':
            export_to_excel(data_df, anomalies, backtest_results, ticker, model_type)
        else:
            print("Análisis terminado. No se ha exportado ningún archivo.")

    else:
        print("No se pudo obtener datos válidos para el análisis.")

if __name__ == "__main__":
    main()


In [ ]:
#Quinto Modelo

In [ ]:
# -*- coding: utf-8 -*-
# Importar las bibliotecas necesarias
import numpy as np
import pandas as pd
import yfinance as yf
import ccxt
from datetime import datetime, timedelta
import ta
import sys
import warnings
from sklearn.ensemble import IsolationForest
from sklearn.svm import OneClassSVM
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestClassifier
import tensorflow as tf
from tensorflow import keras
import plotly.graph_objects as go
import plotly.express as px
from plotly.subplots import make_subplots

# Ignorar advertencias de ta library
warnings.filterwarnings('ignore', category=UserWarning)

def get_real_data(ticker, period='1y'):
    """
    Descarga datos históricos de trading de Yahoo Finance. Si falla, usa una API de intercambio
    de criptomonedas (ccxt) como respaldo.

    Parámetros:
    ticker (str): Símbolo de la acción o criptomoneda (ej. 'AAPL', 'BTC-USD').
    period (str): Período de tiempo para los datos (ej. '1mo', '1y', '5y').

    Retorna:
    pd.DataFrame: DataFrame con los datos de precios y otras características o None si hay un error.
    """
    data = None
    
    # Intento 1: Descargar desde Yahoo Finance con yf.Ticker().history()
    source = 'Yahoo Finance'
    print(f"Intentando descargar datos desde {source} para el ticker: {ticker}...")
    try:
        data = yf.Ticker(ticker).history(period=period, auto_adjust=True)
        
        if data.empty:
            print(f"No se encontraron datos en {source} para el ticker: {ticker}.")
            data = None
        else:
            if isinstance(data.columns, pd.MultiIndex):
                data.columns = data.columns.droplevel()
            
            price_col = None
            for col in ['Close', 'close', 'Adj Close']:
                if col in data.columns:
                    price_col = col
                    break
            
            if not price_col:
                print(f"No se encontró una columna de precios válida en los datos de {source}.")
                data = None
            else:
                data.rename(columns={price_col: 'price', 'Open': 'open', 'High': 'high', 'Low': 'low', 'Volume': 'volume'}, inplace=True)
                data = data[['price', 'open', 'high', 'low', 'volume']]
                print(f"Descarga exitosa desde {source}.")
    except Exception as e:
        print(f"Ocurrió un error con {source}: {e}.")
        data = None

    # Intento 2 (Respaldo): Usar una API de criptomonedas (ccxt)
    if data is None and ticker.endswith('-USD'):
        source = 'ccxt'
        print(f"Intentando descargar datos desde {source} para el ticker: {ticker}...")
        try:
            exchange = ccxt.binance()
            symbol_candidates = [
                ticker.replace('-', '/'),
                ticker.replace('-USD', '/USDT'),
                ticker.split('-')[0] + '/USD',
                ticker.split('-')[0] + '/USDT'
            ]
            
            found_symbol = None
            for symbol in symbol_candidates:
                if symbol in exchange.markets:
                    found_symbol = symbol
                    break
            
            if not found_symbol:
                print(f"El ticker '{ticker}' no se encontró en un formato compatible con {source}.")
            else:
                print(f"Encontrado el símbolo compatible '{found_symbol}'. Descargando datos...")
                since = exchange.parse8601((datetime.now() - timedelta(days=365)).isoformat())
                ohlcv = exchange.fetch_ohlcv(found_symbol, '1d', since)
                
                if not ohlcv:
                    print(f"No se encontraron datos en {source} para el ticker: {found_symbol}.")
                else:
                    data = pd.DataFrame(ohlcv, columns=['timestamp', 'open', 'high', 'low', 'close', 'volume'])
                    data['timestamp'] = pd.to_datetime(data['timestamp'], unit='ms')
                    data.set_index('timestamp', inplace=True)
                    data.rename(columns={'close': 'price'}, inplace=True)
                    data = data[['price', 'open', 'high', 'low', 'volume']]
                    print(f"Descarga exitosa desde {source}.")
        except Exception as e:
            print(f"Ocurrió un error con {source}: {e}")

    if data is not None and not data.empty:
        # Añadir indicadores técnicos para un análisis más completo
        data['RSI'] = ta.momentum.RSIIndicator(data['price']).rsi()
        data['SMA'] = ta.trend.sma_indicator(data['price'], window=20)
        data['ATR'] = ta.volatility.AverageTrueRange(data['high'], data['low'], data['price']).average_true_range()
        # Nuevos indicadores
        data['MACD'] = ta.trend.macd(data['price'])
        data['MACD_Signal'] = ta.trend.macd_signal(data['price'])
        bollinger = ta.volatility.BollingerBands(data['price'])
        data['BBL'] = bollinger.bollinger_lband()
        data['BBM'] = bollinger.bollinger_mavg()
        data['BBU'] = bollinger.bollinger_hband()
        
        # Eliminar las filas con valores NaN (causadas por los indicadores)
        data.dropna(inplace=True)
        
    return data

def optimize_hyperparameters(model_type, data_df):
    """
    Optimiza los hiperparámetros para Isolation Forest o One-Class SVM.

    Parámetros:
    model_type (str): El modelo a optimizar ('IsolationForest' o 'OneClassSVM').
    data_df (pd.DataFrame): DataFrame con los datos de entrada.

    Retorna:
    Un modelo con los mejores parámetros encontrados.
    """
    if model_type == 'IsolationForest':
        model = IsolationForest(random_state=42)
        param_grid = {
            'n_estimators': [50, 100, 200],
            'contamination': [0.01, 0.05, 0.1]
        }
        print("\nOptimizando hiperparámetros para Isolation Forest...")
        grid_search = GridSearchCV(model, param_grid, cv=5, scoring='f1_weighted', verbose=0)
        grid_search.fit(data_df)
        print("Mejores parámetros encontrados: ", grid_search.best_params_)
        return grid_search.best_estimator_
        
    elif model_type == 'OneClassSVM':
        model = OneClassSVM()
        param_grid = {
            'kernel': ['rbf', 'poly', 'sigmoid'],
            'nu': [0.01, 0.05, 0.1],
            'gamma': ['scale', 'auto']
        }
        print("\nOptimizando hiperparámetros para One-Class SVM...")
        grid_search = GridSearchCV(model, param_grid, cv=5, scoring='f1_weighted', verbose=0)
        scaler = MinMaxScaler()
        scaled_data = scaler.fit_transform(data_df)
        grid_search.fit(scaled_data)
        print("Mejores parámetros encontrados: ", grid_search.best_params_)
        return grid_search.best_estimator_
    else:
        return None

def train_autoencoder(X_train):
    """
    Define y entrena un modelo Autoencoder para la detección de anomalías.

    Parámetros:
    X_train (np.array): Datos de entrenamiento.

    Retorna:
    tf.keras.Model: Modelo Autoencoder entrenado.
    """
    input_dim = X_train.shape[1]
    
    model = keras.Sequential([
        keras.layers.Dense(32, activation='relu', input_shape=(input_dim,)),
        keras.layers.Dense(16, activation='relu'),
        keras.layers.Dense(8, activation='relu'),
        keras.layers.Dense(16, activation='relu'),
        keras.layers.Dense(32, activation='relu'),
        keras.layers.Dense(input_dim, activation='sigmoid')
    ])

    model.compile(optimizer='adam', loss='mean_squared_error')
    
    print("Entrenando el modelo Autoencoder...")
    model.fit(X_train, X_train, 
              epochs=50, 
              batch_size=128, 
              validation_split=0.1,
              verbose=0)
    return model

def create_sequences(data, sequence_length):
    """
    Crea secuencias de datos para el entrenamiento y prueba de un modelo LSTM.
    
    Parámetros:
    data (np.array): Datos normalizados.
    sequence_length (int): Longitud de la secuencia de tiempo.

    Retorna:
    tuple: (X, y) donde X son las secuencias e y son los valores a predecir.
    """
    X, y = [], []
    for i in range(len(data) - sequence_length):
        X.append(data[i:(i + sequence_length)])
        y.append(data[i + sequence_length, 0])
    return np.array(X), np.array(y)

def train_lstm_model(X_train, y_train):
    """
    Define y entrena un modelo LSTM para la predicción de series de tiempo.

    Parámetros:
    X_train (np.array): Secuencias de datos de entrenamiento.
    y_train (np.array): Valores de salida correspondientes.

    Retorna:
    tf.keras.Model: Modelo LSTM entrenado.
    """
    sequence_length = X_train.shape[1]
    n_features = X_train.shape[2]
    
    model = keras.Sequential([
        keras.layers.LSTM(64, activation='relu', input_shape=(sequence_length, n_features), return_sequences=True),
        keras.layers.LSTM(32, activation='relu', return_sequences=False),
        keras.layers.Dense(1)
    ])

    model.compile(optimizer='adam', loss='mean_squared_error')
    
    print("Entrenando el modelo LSTM para predicción de series de tiempo...")
    model.fit(X_train, y_train, 
              epochs=50, 
              batch_size=128, 
              validation_split=0.1,
              verbose=0)
    return model

def train_ml_model(data_df):
    """
    Entrena un modelo de Random Forest para predecir la dirección del precio.
    
    Parámetros:
    data_df (pd.DataFrame): DataFrame con los datos y los indicadores.
    
    Retorna:
    sklearn.ensemble.RandomForestClassifier: Modelo entrenado.
    """
    print("\nEntrenando modelo de Machine Learning para señales de trading...")
    
    # Crear la columna objetivo: 1 si el precio sube, 0 si baja o se mantiene
    data_df['target'] = np.where(data_df['price'].shift(-1) > data_df['price'], 1, 0)
    
    # Eliminar la última fila que tendrá NaN
    data_df.dropna(inplace=True)
    
    # Seleccionar las características (features)
    features = ['RSI', 'SMA', 'ATR', 'MACD', 'MACD_Signal', 'BBL', 'BBM', 'BBU']
    X = data_df[features]
    y = data_df['target']
    
    # Dividir los datos en conjuntos de entrenamiento y prueba
    split_index = int(0.8 * len(data_df))
    X_train, X_test = X[:split_index], X[split_index:]
    y_train, y_test = y[:split_index], y[split_index:]
    
    # Inicializar y entrenar el modelo Random Forest
    model = RandomForestClassifier(n_estimators=100, random_state=42)
    model.fit(X_train, y_train)
    
    print("Modelo de Machine Learning entrenado con éxito.")
    return model

def plot_anomalies_interactive(df, predictions, model_type):
    """
    Visualiza los datos y resalta las anomalías detectadas de forma interactiva.
    
    Parámetros:
    df (pd.DataFrame): DataFrame con los datos de precios.
    predictions (np.array): Array con las predicciones del modelo.
    model_type (str): El tipo de modelo utilizado.
    """
    title = f'Detección de Anomalías con {model_type}'
    
    if model_type in ['IsolationForest', 'OneClassSVM']:
        anomalies = df[df[model_type] == -1]
        prediction_label = 'Anomalía Detectada'
    else: # Autoencoder
        threshold = np.mean(predictions) + 2 * np.std(predictions)
        df['anomaly_prediction'] = [1 if error > threshold else 0 for error in predictions]
        anomalies = df[df['anomaly_prediction'] == 1]
        prediction_label = 'Anomalía (Reconstruction Error)'

    # Crear la figura interactiva
    fig = go.Figure()

    # Añadir el rastro del precio normal
    fig.add_trace(go.Scatter(
        x=df.index,
        y=df['price'],
        mode='lines',
        name='Precio (Normal)',
        line=dict(color='blue')
    ))

    # Añadir el rastro de las anomalías
    if not anomalies.empty:
        fig.add_trace(go.Scatter(
            x=anomalies.index,
            y=anomalies['price'],
            mode='markers',
            marker=dict(
                color='red',
                size=10,
                symbol='circle'
            ),
            name=prediction_label,
            hovertext=[f"Fecha: {d.strftime('%Y-%m-%d')}<br>Precio: ${p:.2f}" for d, p in zip(anomalies.index, anomalies['price'])]
        ))

    # Configurar el diseño
    fig.update_layout(
        title=title,
        xaxis_title='Fecha',
        yaxis_title='Precio',
        template='plotly_dark',
        hovermode='x unified'
    )

    fig.show()

def plot_lstm_anomalies_interactive(df, predictions, scaler):
    """
    Visualiza los datos y resalta las anomalías de predicción detectadas por el modelo LSTM.

    Parámetros:
    df (pd.DataFrame): DataFrame con los datos de precios.
    predictions (np.array): Array de predicciones del modelo LSTM.
    scaler (MinMaxScaler): El scaler utilizado para normalizar los datos.
    """
    df_plot = df.iloc[len(df) - len(predictions):].copy()
    
    num_features = df.shape[1]
    predictions_denormalized = scaler.inverse_transform(np.c_[predictions, np.zeros((len(predictions), num_features - 1))])[:, 0]
    
    df_plot['predicted_price'] = predictions_denormalized
    df_plot['prediction_error'] = np.abs(df_plot['price'] - df_plot['predicted_price'])
    
    threshold = np.mean(df_plot['prediction_error']) + 2 * np.std(df_plot['prediction_error'])
    df_plot['anomaly'] = df_plot['prediction_error'] > threshold
    
    anomalies = df_plot[df_plot['anomaly']]

    # Crear la figura interactiva
    fig = go.Figure()

    # Añadir el rastro del precio real
    fig.add_trace(go.Scatter(
        x=df_plot.index,
        y=df_plot['price'],
        mode='lines',
        name='Precio Real',
        line=dict(color='blue')
    ))

    # Añadir el rastro del precio predicho
    fig.add_trace(go.Scatter(
        x=df_plot.index,
        y=df_plot['predicted_price'],
        mode='lines',
        name='Precio Predicho (LSTM)',
        line=dict(color='green', dash='dash')
    ))
    
    # Añadir los marcadores de anomalías
    if not anomalies.empty:
        fig.add_trace(go.Scatter(
            x=anomalies.index,
            y=anomalies['price'],
            mode='markers',
            name='Anomalía de Predicción',
            marker=dict(symbol='circle', color='red', size=10),
            hovertext=[f"Fecha: {d.strftime('%Y-%m-%d')}<br>Precio: ${p:.2f}" for d, p in zip(anomalies.index, anomalies['price'])]
        ))

    # Configurar el diseño
    fig.update_layout(
        title='Detección de Anomalías Futuras con LSTM',
        xaxis_title='Fecha',
        yaxis_title='Precio',
        template='plotly_dark',
        hovermode='x unified'
    )
    
    fig.show()


def generate_recommendation(anomalies, data_df):
    """
    Genera una recomendación de trading basada en la última anomalía detectada
    y los indicadores del último día.

    Parámetros:
    anomalies (pd.DataFrame): DataFrame con los puntos anómalos.
    data_df (pd.DataFrame): DataFrame con todos los datos.

    Retorna:
    tuple: (str, str) con el resumen y la recomendación.
    """
    if anomalies.empty:
        summary = "El análisis no ha detectado anomalías significativas en los datos recientes."
        recommendation = "MANTENER"
        reason = "El mercado parece estable. No se han identificado movimientos de precio inusuales."
        return summary, recommendation, reason

    latest_anomaly = anomalies.iloc[-1]
    last_data = data_df.iloc[-1]

    # Lógica mejorada que considera la dirección de la anomalía y el volumen
    summary = f"El modelo ha detectado una anomalía reciente el {latest_anomaly.name.strftime('%Y-%m-%d')}."
    
    # Evaluar la señal de trading más reciente
    if last_data['MACD'] > last_data['MACD_Signal'] and last_data['MACD'] > 0:
        recommendation = "COMPRAR"
        reason = "La línea MACD ha cruzado por encima de la línea de señal y está en territorio positivo, lo que indica un fuerte impulso alcista."
    elif last_data['MACD'] < last_data['MACD_Signal'] and last_data['MACD'] < 0:
        recommendation = "VENDER"
        reason = "La línea MACD ha cruzado por debajo de la línea de señal y está en territorio negativo, lo que indica un fuerte impulso bajista."
    else:
        recommendation = "MANTENER"
        reason = "Los indicadores no muestran una señal clara. El mercado parece estar en consolidación."

    return summary, recommendation, reason

def run_backtest(df, strategy_type, ml_model=None):
    """
    Simula una estrategia de trading. Puede ser basada en el cruce de MACD o en un modelo de ML.
    
    Parámetros:
    df (pd.DataFrame): DataFrame con los datos y los indicadores.
    strategy_type (str): 'MACD' o 'ML'.
    ml_model (RandomForestClassifier): El modelo de ML para la estrategia 'ML'.

    Retorna:
    pd.DataFrame: DataFrame con los resultados del backtest.
    """
    print(f"\n--- Ejecutando Backtest con Estrategia: {strategy_type} ---")
    portfolio = pd.DataFrame(index=df.index)
    portfolio['asset_price'] = df['price']
    portfolio['holdings'] = 0.0
    portfolio['cash'] = 1000.0  # Capital inicial
    portfolio['total_value'] = portfolio['cash']
    
    in_position = False
    buy_price = 0
    trades = []
    
    stop_loss_percent = 0.05 # 5% de stop-loss

    features = ['RSI', 'SMA', 'ATR', 'MACD', 'MACD_Signal', 'BBL', 'BBM', 'BBU']
    
    for i in range(1, len(portfolio)):
        portfolio.loc[portfolio.index[i], 'cash'] = portfolio.loc[portfolio.index[i-1], 'cash']
        portfolio.loc[portfolio.index[i], 'holdings'] = portfolio.loc[portfolio.index[i-1], 'holdings']
        
        signal = 0 # 0 = Mantener, 1 = Comprar, -1 = Vender
        
        # Lógica de señales
        if strategy_type == 'MACD':
            # Señal de compra: MACD cruza por encima de la línea de señal
            if df['MACD'].iloc[i] > df['MACD_Signal'].iloc[i] and df['MACD'].iloc[i-1] <= df['MACD_Signal'].iloc[i-1]:
                signal = 1
            # Señal de venta: MACD cruza por debajo de la línea de señal
            elif df['MACD'].iloc[i] < df['MACD_Signal'].iloc[i] and df['MACD'].iloc[i-1] >= df['MACD_Signal'].iloc[i-1]:
                signal = -1
        
        elif strategy_type == 'ML':
            # Predecir la señal con el modelo de ML
            if i >= 10: # Asegurarse de tener suficientes datos para las características
                data_point = df[features].iloc[i].values.reshape(1, -1)
                prediction = ml_model.predict(data_point)[0]
                if prediction == 1:
                    signal = 1
                elif prediction == 0:
                    # Usar la señal de MACD como respaldo si el ML predice no subir
                    if df['MACD'].iloc[i] < df['MACD_Signal'].iloc[i] and df['MACD'].iloc[i-1] >= df['MACD_Signal'].iloc[i-1]:
                        signal = -1
        
        # Ejecución de la estrategia
        # Comprar
        if signal == 1 and not in_position:
            if portfolio.loc[portfolio.index[i], 'cash'] > 0:
                shares_to_buy = portfolio.loc[portfolio.index[i], 'cash'] / portfolio.loc[portfolio.index[i], 'asset_price']
                portfolio.loc[portfolio.index[i], 'holdings'] += shares_to_buy
                portfolio.loc[portfolio.index[i], 'cash'] = 0.0
                in_position = True
                buy_price = portfolio.loc[portfolio.index[i], 'asset_price']
                trades.append({'date': portfolio.index[i], 'type': 'buy', 'price': buy_price})
                print(f"COMPRA: el {df.index[i].strftime('%Y-%m-%d')}, precio: ${df['price'].iloc[i]:.2f}")
        
        # Vender
        elif signal == -1 and in_position:
            cash_from_sale = portfolio.loc[portfolio.index[i], 'holdings'] * portfolio.loc[portfolio.index[i], 'asset_price']
            portfolio.loc[portfolio.index[i], 'cash'] = cash_from_sale
            portfolio.loc[portfolio.index[i], 'holdings'] = 0.0
            in_position = False
            trades.append({'date': portfolio.index[i], 'type': 'sell', 'price': portfolio.loc[portfolio.index[i], 'asset_price']})
            print(f"VENTA: el {df.index[i].strftime('%Y-%m-%d')}, precio: ${df['price'].iloc[i]:.2f}")
        
        # Gestión de Riesgos: Stop-Loss
        elif in_position and (portfolio.loc[portfolio.index[i], 'asset_price'] / buy_price - 1) < -stop_loss_percent:
            print(f"STOP-LOSS ACTIVADO: el {df.index[i].strftime('%Y-%m-%d')}, precio: ${df['price'].iloc[i]:.2f}")
            cash_from_sale = portfolio.loc[portfolio.index[i], 'holdings'] * portfolio.loc[portfolio.index[i], 'asset_price']
            portfolio.loc[portfolio.index[i], 'cash'] = cash_from_sale
            portfolio.loc[portfolio.index[i], 'holdings'] = 0.0
            in_position = False
            trades.append({'date': portfolio.index[i], 'type': 'stop_loss', 'price': portfolio.loc[portfolio.index[i], 'asset_price']})
            
        portfolio.loc[portfolio.index[i], 'total_value'] = portfolio.loc[portfolio.index[i], 'cash'] + portfolio.loc[portfolio.index[i], 'holdings'] * portfolio.loc[portfolio.index[i], 'asset_price']

    # Cierre de la última posición si sigue abierta al final
    if in_position:
        cash_from_sale = portfolio.iloc[-1]['holdings'] * portfolio.iloc[-1]['asset_price']
        portfolio.iloc[-1]['cash'] = cash_from_sale
        portfolio.iloc[-1]['holdings'] = 0.0
        portfolio.iloc[-1]['total_value'] = portfolio.iloc[-1]['cash']
        trades.append({'date': portfolio.index[-1], 'type': 'sell', 'price': portfolio.iloc[-1]['asset_price']})

    # Calcular métricas de rendimiento
    initial_value = portfolio['total_value'].iloc[0]
    final_value = portfolio['total_value'].iloc[-1]
    
    returns = portfolio['total_value'].pct_change().dropna()
    sharpe_ratio = np.sqrt(252) * returns.mean() / returns.std()
    
    max_drawdown = (portfolio['total_value'].cummax() - portfolio['total_value']).max() / portfolio['total_value'].cummax().max()
    
    winning_trades = 0
    losing_trades = 0
    
    buy_prices = [trade['price'] for trade in trades if trade['type'] == 'buy']
    sell_prices = [trade['price'] for trade in trades if trade['type'] in ['sell', 'stop_loss']]
    
    # Simple win rate calculation
    if len(buy_prices) == len(sell_prices) and len(buy_prices) > 0:
        for i in range(len(buy_prices)):
            if sell_prices[i] > buy_prices[i]:
                winning_trades += 1
            else:
                losing_trades += 1
    
    total_trades = winning_trades + losing_trades
    win_rate = winning_trades / total_trades if total_trades > 0 else 0

    print("\n--- Resultados del Backtest ---")
    print(f"Valor inicial del portafolio: ${initial_value:.2f}")
    print(f"Valor final del portafolio: ${final_value:.2f}")
    
    backtest_return = (final_value - initial_value) / initial_value
    market_return = (df['price'].iloc[-1] - df['price'].iloc[0]) / df['price'].iloc[0]
    
    print(f"Retorno del Backtest: {backtest_return * 100:.2f}%")
    print(f"Retorno del mercado: {market_return * 100:.2f}%")
    print(f"Sharpe Ratio: {sharpe_ratio:.2f}")
    print(f"Máximo Drawdown: {max_drawdown * 100:.2f}%")
    print(f"Tasa de Éxito (Win Rate): {win_rate * 100:.2f}%")

    # Nueva llamada a la función de graficación interactiva
    plot_backtest_interactive(portfolio, df, trades, strategy_type, initial_value)

    return portfolio

def plot_backtest_interactive(portfolio, df, trades, strategy_type, initial_value):
    """
    Visualiza el rendimiento del backtest vs. el mercado de forma interactiva.

    Parámetros:
    portfolio (pd.DataFrame): DataFrame con el valor del portafolio.
    df (pd.DataFrame): DataFrame original con el precio del activo.
    trades (list): Lista de diccionarios de operaciones de trading.
    strategy_type (str): El tipo de estrategia.
    initial_value (float): El valor inicial del portafolio.
    """
    
    fig = go.Figure()

    # Añadir el rastro del valor del portafolio
    fig.add_trace(go.Scatter(
        x=portfolio.index,
        y=portfolio['total_value'],
        mode='lines',
        name='Valor del Portafolio',
        line=dict(color='purple')
    ))

    # Añadir el rastro del rendimiento del mercado
    fig.add_trace(go.Scatter(
        x=df.index,
        y=(df['price'] / df['price'].iloc[0]) * initial_value,
        mode='lines',
        name='Rendimiento del Mercado',
        line=dict(color='orange', dash='dash')
    ))

    # Añadir los marcadores de operaciones
    buy_dates = [t['date'] for t in trades if t['type'] == 'buy']
    buy_prices = [t['price'] for t in trades if t['type'] == 'buy']
    sell_dates = [t['date'] for t in trades if t['type'] == 'sell']
    sell_prices = [t['price'] for t in trades if t['type'] == 'sell']
    sl_dates = [t['date'] for t in trades if t['type'] == 'stop_loss']
    sl_prices = [t['price'] for t in trades if t['type'] == 'stop_loss']

    fig.add_trace(go.Scatter(
        x=buy_dates,
        y=buy_prices,
        mode='markers',
        name='Compra',
        marker=dict(symbol='triangle-up', color='green', size=10),
        hovertext=[f"Compra<br>Fecha: {d.strftime('%Y-%m-%d')}<br>Precio: ${p:.2f}" for d, p in zip(buy_dates, buy_prices)],
        hoverinfo='text'
    ))

    fig.add_trace(go.Scatter(
        x=sell_dates,
        y=sell_prices,
        mode='markers',
        name='Venta',
        marker=dict(symbol='triangle-down', color='red', size=10),
        hovertext=[f"Venta<br>Fecha: {d.strftime('%Y-%m-%d')}<br>Precio: ${p:.2f}" for d, p in zip(sell_dates, sell_prices)],
        hoverinfo='text'
    ))
    
    fig.add_trace(go.Scatter(
        x=sl_dates,
        y=sl_prices,
        mode='markers',
        name='Stop-Loss',
        marker=dict(symbol='x', color='darkred', size=10),
        hovertext=[f"Stop-Loss<br>Fecha: {d.strftime('%Y-%m-%d')}<br>Precio: ${p:.2f}" for d, p in zip(sl_dates, sl_prices)],
        hoverinfo='text'
    ))

    # Configurar el diseño
    fig.update_layout(
        title=f'Rendimiento del Backtest vs. Mercado ({strategy_type} Strategy)',
        xaxis_title='Fecha',
        yaxis_title='Valor del Portafolio ($)',
        template='plotly_dark',
        hovermode='x unified'
    )

    fig.show()


def export_to_excel(df, anomalies, backtest_results, ticker, model_type):
    """
    Exporta el DataFrame completo, un resumen de las anomalías y los resultados del backtest
    a un archivo de Excel.

    Parámetros:
    df (pd.DataFrame): DataFrame con todos los datos.
    anomalies (pd.DataFrame): DataFrame con las anomalías detectadas.
    backtest_results (pd.DataFrame): DataFrame con los resultados del backtest.
    ticker (str): Ticker del activo.
    model_type (str): El modelo utilizado.
    """
    try:
        filename = f"Analisis_Anomalias_{ticker}_{model_type}.xlsx"
        writer = pd.ExcelWriter(filename, engine='xlsxwriter')
        
        df.to_excel(writer, sheet_name='Datos Completos')
        
        if not anomalies.empty:
            anomalies.to_excel(writer, sheet_name='Anomalias Detectadas')
        else:
            empty_df = pd.DataFrame(["No se detectaron anomalías."], columns=["Resultado"])
            empty_df.to_excel(writer, sheet_name='Anomalias Detectadas')
        
        if not backtest_results.empty:
            backtest_results.to_excel(writer, sheet_name='Resultados Backtest')
        
        writer._save()
        print(f"\n¡Análisis exportado a '{filename}' con éxito!")
    except Exception as e:
        print(f"Error al exportar a Excel: {e}")

def main():
    """
    Función principal para ejecutar el detector de anomalías.
    """
    ticker = input("Introduce el ticker de la acción o criptomoneda (ej. AAPL, BTC-USD) según Yahoo Finance. Si la descarga falla, buscaremos automáticamente en otra fuente: ").upper()
    
    data_df = get_real_data(ticker)
    
    if data_df is not None and not data_df.empty:
        print("\n--- Modelos de Detección de Anomalías ---")
        print("1. Isolation Forest (modelo más simple y rápido)")
        print("2. Autoencoder (modelo de IA avanzado)")
        print("3. LSTM (para predicción y anomalías futuras)")
        print("4. One-Class SVM (para patrones complejos)")
        model_choice = input("Elige el modelo a usar (1, 2, 3 o 4): ")
        
        anomalies = pd.DataFrame() # DataFrame vacío por defecto
        
        if model_choice == '1':
            model_type = 'IsolationForest'
            model = optimize_hyperparameters(model_type, data_df)
            predictions = model.fit_predict(data_df)
            data_df[model_type] = predictions
            anomalies = data_df[data_df[model_type] == -1]
            print(f"Número de anomalías detectadas: {(predictions == -1).sum()}")
            plot_anomalies_interactive(data_df, predictions, model_type)
        
        elif model_choice == '2':
            model_type = 'Autoencoder'
            scaler = MinMaxScaler()
            scaled_data = scaler.fit_transform(data_df)
            
            autoencoder = train_autoencoder(scaled_data)
            
            reconstructions = autoencoder.predict(scaled_data)
            mse = np.mean(np.power(scaled_data - reconstructions, 2), axis=1)
            
            threshold = np.mean(mse) + 2 * np.std(mse)
            data_df['anomaly'] = [True if error > threshold else False for error in mse]
            anomalies = data_df[data_df['anomaly']]
            print(f"Número de anomalías detectadas: {anomalies.shape[0]}")
            plot_anomalies_interactive(data_df, mse, model_type)

        elif model_choice == '3':
            model_type = 'LSTM'
            sequence_length = 30
            scaler = MinMaxScaler()
            scaled_data = scaler.fit_transform(data_df)
            X, y = create_sequences(scaled_data, sequence_length)
            
            test_size = int(len(X) * 0.2)
            X_train, y_train = X[:-test_size], y[:-test_size]
            X_test, y_test = X[-test_size:], y[-test_size:]
            
            lstm_model = train_lstm_model(X_train, y_train)
            
            predictions = lstm_model.predict(X_test)
            
            data_df_plot = data_df.iloc[len(data_df) - len(predictions):].copy()
            
            num_features = data_df.shape[1]
            predictions_denormalized = scaler.inverse_transform(np.c_[predictions, np.zeros((len(predictions), num_features - 1))])[:, 0]
            data_df_plot['predicted_price'] = predictions_denormalized
            data_df_plot['prediction_error'] = np.abs(data_df_plot['price'] - data_df_plot['predicted_price'])
            
            threshold = np.mean(data_df_plot['prediction_error']) + 2 * np.std(data_df_plot['prediction_error'])
            data_df_plot['anomaly'] = data_df_plot['prediction_error'] > threshold
            anomalies = data_df_plot[data_df_plot['anomaly']]
            print(f"Número de anomalías de predicción detectadas: {anomalies.shape[0]}")
            plot_lstm_anomalies_interactive(data_df, predictions, scaler)

        elif model_choice == '4':
            model_type = 'OneClassSVM'
            model = optimize_hyperparameters(model_type, data_df)
            scaler = MinMaxScaler()
            scaled_data = scaler.fit_transform(data_df)
            predictions = model.fit_predict(scaled_data)
            data_df[model_type] = predictions
            anomalies = data_df[data_df[model_type] == -1]
            print(f"Número de anomalías detectadas: {(predictions == -1).sum()}")
            plot_anomalies_interactive(data_df, predictions, model_type)

        else:
            print("Elección de modelo inválida. Por favor, reinicia el script.")
            sys.exit()

        print("\n--- Resumen del Análisis ---")
        summary, recommendation, reason = generate_recommendation(anomalies, data_df)
        print(f"Resumen: {summary}")
        print(f"Recomendación: {recommendation}")
        print(f"Motivo: {reason}")
        print("\n*Este análisis es solo para fines informativos y no debe ser considerado un consejo financiero.*")
        
        backtest_results = pd.DataFrame()
        backtest_choice = input("\n¿Deseas realizar un backtest de la estrategia? (s/n): ").lower()
        if backtest_choice == 's':
            print("\n--- Opciones de Backtesting ---")
            print("1. Estrategia de Cruce MACD")
            print("2. Estrategia basada en Machine Learning")
            strategy_choice = input("Elige la estrategia para el backtest (1 o 2): ")
            
            if strategy_choice == '1':
                backtest_results = run_backtest(data_df, 'MACD')
            elif strategy_choice == '2':
                ml_model = train_ml_model(data_df.copy())
                # Asegurarse de que el DataFrame del backtest no tenga la columna 'target'
                df_for_backtest = data_df.drop(columns=['target'], errors='ignore')
                backtest_results = run_backtest(df_for_backtest, 'ML', ml_model)
            else:
                print("Elección de estrategia inválida. No se realizará el backtest.")

        export_choice = input("\n¿Deseas exportar los resultados a un archivo de Excel? (s/n): ").lower()
        if export_choice == 's':
            export_to_excel(data_df, anomalies, backtest_results, ticker, model_type)
        else:
            print("Análisis terminado. No se ha exportado ningún archivo.")

    else:
        print("No se pudo obtener datos válidos para el análisis.")

if __name__ == "__main__":
    main()


In [ ]:
#SEXTO MODELO

In [ ]:
# -*- coding: utf-8 -*-
# Importar las bibliotecas necesarias
import numpy as np
import pandas as pd
import yfinance as yf
import ccxt
from datetime import datetime, timedelta
import ta
import sys
import warnings
from sklearn.ensemble import IsolationForest
from sklearn.svm import OneClassSVM
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
import tensorflow as tf
from tensorflow import keras
import plotly.graph_objects as go
import plotly.express as px
from plotly.subplots import make_subplots

# Ignorar advertencias de ta library
warnings.filterwarnings('ignore', category=UserWarning)

# Configuración global para el backtest
BACKTEST_CONFIG = {
    'initial_cash': 1000.0,
    'stop_loss_percent': 0.05,
    'transaction_cost_percent': 0.001 # 0.1% de comisión por operación
}

def get_real_data(ticker, period='1y'):
    """
    Descarga datos históricos de trading de Yahoo Finance. Si falla, usa una API de intercambio
    de criptomonedas (ccxt) como respaldo.

    Parámetros:
    ticker (str): Símbolo de la acción o criptomoneda (ej. 'AAPL', 'BTC-USD').
    period (str): Período de tiempo para los datos (ej. '1mo', '1y', '5y').

    Retorna:
    pd.DataFrame: DataFrame con los datos de precios y otras características o None si hay un error.
    """
    data = None
    
    # Intento 1: Descargar desde Yahoo Finance con yf.Ticker().history()
    source = 'Yahoo Finance'
    print(f"Intentando descargar datos desde {source} para el ticker: {ticker}...")
    try:
        data = yf.Ticker(ticker).history(period=period, auto_adjust=True)
        
        if data.empty:
            print(f"No se encontraron datos en {source} para el ticker: {ticker}.")
            data = None
        else:
            if isinstance(data.columns, pd.MultiIndex):
                data.columns = data.columns.droplevel()
            
            price_col = None
            for col in ['Close', 'close', 'Adj Close']:
                if col in data.columns:
                    price_col = col
                    break
            
            if not price_col:
                print(f"No se encontró una columna de precios válida en los datos de {source}.")
                data = None
            else:
                data.rename(columns={price_col: 'price', 'Open': 'open', 'High': 'high', 'Low': 'low', 'Volume': 'volume'}, inplace=True)
                data = data[['price', 'open', 'high', 'low', 'volume']]
                print(f"Descarga exitosa desde {source}.")
    except Exception as e:
        print(f"Ocurrió un error con {source}: {e}.")
        data = None

    # Intento 2 (Respaldo): Usar una API de criptomonedas (ccxt)
    if data is None and ticker.endswith('-USD'):
        source = 'ccxt'
        print(f"Intentando descargar datos desde {source} para el ticker: {ticker}...")
        try:
            exchange = ccxt.binance()
            symbol_candidates = [
                ticker.replace('-', '/'),
                ticker.replace('-USD', '/USDT'),
                ticker.split('-')[0] + '/USD',
                ticker.split('-')[0] + '/USDT'
            ]
            
            found_symbol = None
            for symbol in symbol_candidates:
                if symbol in exchange.markets:
                    found_symbol = symbol
                    break
            
            if not found_symbol:
                print(f"El ticker '{ticker}' no se encontró en un formato compatible con {source}.")
            else:
                print(f"Encontrado el símbolo compatible '{found_symbol}'. Descargando datos...")
                since = exchange.parse8601((datetime.now() - timedelta(days=365)).isoformat())
                ohlcv = exchange.fetch_ohlcv(found_symbol, '1d', since)
                
                if not ohlcv:
                    print(f"No se encontraron datos en {source} para el ticker: {found_symbol}.")
                else:
                    data = pd.DataFrame(ohlcv, columns=['timestamp', 'open', 'high', 'low', 'close', 'volume'])
                    data['timestamp'] = pd.to_datetime(data['timestamp'], unit='ms')
                    data.set_index('timestamp', inplace=True)
                    data.rename(columns={'close': 'price'}, inplace=True)
                    data = data[['price', 'open', 'high', 'low', 'volume']]
                    print(f"Descarga exitosa desde {source}.")
        except Exception as e:
            print(f"Ocurrió un error con {source}: {e}")

    if data is not None and not data.empty:
        # Añadir indicadores técnicos para un análisis más completo
        data['RSI'] = ta.momentum.RSIIndicator(data['price']).rsi()
        data['SMA'] = ta.trend.sma_indicator(data['price'], window=20)
        data['ATR'] = ta.volatility.AverageTrueRange(data['high'], data['low'], data['price']).average_true_range()
        # Nuevos indicadores
        data['MACD'] = ta.trend.macd(data['price'])
        data['MACD_Signal'] = ta.trend.macd_signal(data['price'])
        data['MACD_Hist'] = ta.trend.macd_diff(data['price'])
        bollinger = ta.volatility.BollingerBands(data['price'])
        data['BBL'] = bollinger.bollinger_lband()
        data['BBM'] = bollinger.bollinger_mavg()
        data['BBU'] = bollinger.bollinger_hband()
        
        # Eliminar las filas con valores NaN (causadas por los indicadores)
        data.dropna(inplace=True)
        
    return data

def optimize_hyperparameters(model_type, data_df):
    """
    Optimiza los hiperparámetros para Isolation Forest, One-Class SVM o Gradient Boosting.

    Parámetros:
    model_type (str): El modelo a optimizar ('IsolationForest', 'OneClassSVM' o 'GradientBoosting').
    data_df (pd.DataFrame): DataFrame con los datos de entrada.

    Retorna:
    Un modelo con los mejores parámetros encontrados.
    """
    if model_type == 'IsolationForest':
        model = IsolationForest(random_state=42)
        param_grid = {
            'n_estimators': [50, 100, 200],
            'contamination': [0.01, 0.05, 0.1]
        }
        print("\nOptimizando hiperparámetros para Isolation Forest...")
        grid_search = GridSearchCV(model, param_grid, cv=5, scoring='f1_weighted', verbose=0)
        grid_search.fit(data_df)
        print("Mejores parámetros encontrados: ", grid_search.best_params_)
        return grid_search.best_estimator_
        
    elif model_type == 'OneClassSVM':
        model = OneClassSVM()
        param_grid = {
            'kernel': ['rbf', 'poly', 'sigmoid'],
            'nu': [0.01, 0.05, 0.1],
            'gamma': ['scale', 'auto']
        }
        print("\nOptimizando hiperparámetros para One-Class SVM...")
        grid_search = GridSearchCV(model, param_grid, cv=5, scoring='f1_weighted', verbose=0)
        scaler = MinMaxScaler()
        scaled_data = scaler.fit_transform(data_df)
        grid_search.fit(scaled_data)
        print("Mejores parámetros encontrados: ", grid_search.best_params_)
        return grid_search.best_estimator_

    elif model_type == 'GradientBoosting':
        model = GradientBoostingClassifier(random_state=42)
        param_grid = {
            'n_estimators': [50, 100, 200],
            'learning_rate': [0.01, 0.1, 0.2]
        }
        print("\nOptimizando hiperparámetros para Gradient Boosting...")
        grid_search = GridSearchCV(model, param_grid, cv=5, scoring='accuracy', verbose=0)
        # Preparar los datos para el entrenamiento
        data_df['target'] = np.where(data_df['price'].shift(-1) > data_df['price'], 1, 0)
        data_df.dropna(inplace=True)
        features = ['RSI', 'SMA', 'ATR', 'MACD', 'MACD_Signal', 'BBL', 'BBM', 'BBU']
        X = data_df[features]
        y = data_df['target']
        grid_search.fit(X, y)
        print("Mejores parámetros encontrados: ", grid_search.best_params_)
        return grid_search.best_estimator_

    else:
        return None

def train_autoencoder(X_train):
    """
    Define y entrena un modelo Autoencoder para la detección de anomalías.

    Parámetros:
    X_train (np.array): Datos de entrenamiento.

    Retorna:
    tf.keras.Model: Modelo Autoencoder entrenado.
    """
    input_dim = X_train.shape[1]
    
    model = keras.Sequential([
        keras.layers.Dense(32, activation='relu', input_shape=(input_dim,)),
        keras.layers.Dense(16, activation='relu'),
        keras.layers.Dense(8, activation='relu'),
        keras.layers.Dense(16, activation='relu'),
        keras.layers.Dense(32, activation='relu'),
        keras.layers.Dense(input_dim, activation='sigmoid')
    ])

    model.compile(optimizer='adam', loss='mean_squared_error')
    
    print("Entrenando el modelo Autoencoder...")
    model.fit(X_train, X_train, 
              epochs=50, 
              batch_size=128, 
              validation_split=0.1,
              verbose=0)
    return model

def create_sequences(data, sequence_length):
    """
    Crea secuencias de datos para el entrenamiento y prueba de un modelo LSTM.
    
    Parámetros:
    data (np.array): Datos normalizados.
    sequence_length (int): Longitud de la secuencia de tiempo.

    Retorna:
    tuple: (X, y) donde X son las secuencias e y son los valores a predecir.
    """
    X, y = [], []
    for i in range(len(data) - sequence_length):
        X.append(data[i:(i + sequence_length)])
        y.append(data[i + sequence_length, 0])
    return np.array(X), np.array(y)

def train_lstm_model(X_train, y_train):
    """
    Define y entrena un modelo LSTM para la predicción de series de tiempo.

    Parámetros:
    X_train (np.array): Secuencias de datos de entrenamiento.
    y_train (np.array): Valores de salida correspondientes.

    Retorna:
    tf.keras.Model: Modelo LSTM entrenado.
    """
    sequence_length = X_train.shape[1]
    n_features = X_train.shape[2]
    
    model = keras.Sequential([
        keras.layers.LSTM(64, activation='relu', input_shape=(sequence_length, n_features), return_sequences=True),
        keras.layers.LSTM(32, activation='relu', return_sequences=False),
        keras.layers.Dense(1)
    ])

    model.compile(optimizer='adam', loss='mean_squared_error')
    
    print("Entrenando el modelo LSTM para predicción de series de tiempo...")
    model.fit(X_train, y_train, 
              epochs=50, 
              batch_size=128, 
              validation_split=0.1,
              verbose=0)
    return model

def train_ml_model(data_df, model_type):
    """
    Entrena un modelo de Random Forest o Gradient Boosting para predecir la dirección del precio.
    
    Parámetros:
    data_df (pd.DataFrame): DataFrame con los datos y los indicadores.
    model_type (str): 'RandomForest' o 'GradientBoosting'.
    
    Retorna:
    sklearn.ensemble.Classifier: Modelo entrenado.
    """
    print(f"\nEntrenando modelo de Machine Learning ({model_type}) para señales de trading...")
    
    # Crear la columna objetivo: 1 si el precio sube, 0 si baja o se mantiene
    data_df['target'] = np.where(data_df['price'].shift(-1) > data_df['price'], 1, 0)
    
    # Eliminar la última fila que tendrá NaN
    data_df.dropna(inplace=True)
    
    # Seleccionar las características (features)
    features = ['RSI', 'SMA', 'ATR', 'MACD', 'MACD_Signal', 'MACD_Hist', 'BBL', 'BBM', 'BBU']
    X = data_df[features]
    y = data_df['target']
    
    # Dividir los datos en conjuntos de entrenamiento y prueba
    split_index = int(0.8 * len(data_df))
    X_train, X_test = X[:split_index], X[split_index:]
    y_train, y_test = y[:split_index], y[split_index:]
    
    # Inicializar y entrenar el modelo
    if model_type == 'RandomForest':
        model = RandomForestClassifier(n_estimators=100, random_state=42)
    elif model_type == 'GradientBoosting':
        model = optimize_hyperparameters('GradientBoosting', data_df)
    else:
        print("Tipo de modelo ML no válido.")
        return None

    model.fit(X_train, y_train)
    
    print("Modelo de Machine Learning entrenado con éxito.")
    return model


def plot_interactive_analysis(df, anomalies=None, model_type=None):
    """
    Visualiza los datos de precios, indicadores y anomalías de forma interactiva.
    """
    # Crear subplots con dos filas y un espacio para los indicadores
    fig = make_subplots(
        rows=3, cols=1, 
        shared_xaxes=True, 
        vertical_spacing=0.1,
        row_heights=[0.5, 0.25, 0.25],
        subplot_titles=('Precio del Activo', 'RSI', 'MACD'))

    # Gráfico de velas para el precio
    fig.add_trace(go.Candlestick(x=df.index,
                                 open=df['open'],
                                 high=df['high'],
                                 low=df['low'],
                                 close=df['price'],
                                 name='Precio',
                                 increasing_line_color='green', 
                                 decreasing_line_color='red'),
                  row=1, col=1)

    # Añadir los marcadores de anomalías
    if anomalies is not None and not anomalies.empty:
        fig.add_trace(go.Scatter(
            x=anomalies.index,
            y=anomalies['price'],
            mode='markers',
            marker=dict(
                color='red',
                size=10,
                symbol='circle'
            ),
            name='Anomalía Detectada',
            hovertext=[f"Fecha: {d.strftime('%Y-%m-%d')}<br>Precio: ${p:.2f}" for d, p in zip(anomalies.index, anomalies['price'])],
            hoverinfo='text'
        ), row=1, col=1)

    # Gráfico de RSI
    fig.add_trace(go.Scatter(x=df.index, y=df['RSI'], name='RSI', line=dict(color='orange')), row=2, col=1)
    fig.add_hline(y=70, line_dash="dash", line_color="red", row=2, col=1)
    fig.add_hline(y=30, line_dash="dash", line_color="green", row=2, col=1)

    # Gráfico de MACD
    colors = np.where(df['MACD_Hist'] > 0, 'green', 'red')
    fig.add_trace(go.Bar(x=df.index, y=df['MACD_Hist'], name='MACD Histograma', marker_color=colors), row=3, col=1)
    fig.add_trace(go.Scatter(x=df.index, y=df['MACD'], name='MACD', line=dict(color='blue')), row=3, col=1)
    fig.add_trace(go.Scatter(x=df.index, y=df['MACD_Signal'], name='Señal MACD', line=dict(color='orange')), row=3, col=1)
    
    # Configurar el diseño
    fig.update_layout(
        title_text=f'Análisis de Trading ({model_type or "sin modelo"})',
        xaxis_rangeslider_visible=False,
        template='plotly_dark',
        hovermode='x unified',
        height=700
    )
    fig.show()


def plot_lstm_anomalies_interactive(df, predictions, scaler):
    """
    Visualiza los datos y resalta las anomalías de predicción detectadas por el modelo LSTM.

    Parámetros:
    df (pd.DataFrame): DataFrame con los datos de precios.
    predictions (np.array): Array de predicciones del modelo LSTM.
    scaler (MinMaxScaler): El scaler utilizado para normalizar los datos.
    """
    df_plot = df.iloc[len(df) - len(predictions):].copy()
    
    num_features = df.shape[1]
    predictions_denormalized = scaler.inverse_transform(np.c_[predictions, np.zeros((len(predictions), num_features - 1))])[:, 0]
    
    df_plot['predicted_price'] = predictions_denormalized
    df_plot['prediction_error'] = np.abs(df_plot['price'] - df_plot['predicted_price'])
    
    threshold = np.mean(df_plot['prediction_error']) + 2 * np.std(df_plot['prediction_error'])
    df_plot['anomaly'] = df_plot['prediction_error'] > threshold
    
    anomalies = df_plot[df_plot['anomaly']]

    # Crear la figura interactiva
    fig = go.Figure()

    # Añadir el rastro del precio real
    fig.add_trace(go.Scatter(
        x=df_plot.index,
        y=df_plot['price'],
        mode='lines',
        name='Precio Real',
        line=dict(color='blue')
    ))

    # Añadir el rastro del precio predicho
    fig.add_trace(go.Scatter(
        x=df_plot.index,
        y=df_plot['predicted_price'],
        mode='lines',
        name='Precio Predicho (LSTM)',
        line=dict(color='green', dash='dash')
    ))
    
    # Añadir los marcadores de anomalías
    if not anomalies.empty:
        fig.add_trace(go.Scatter(
            x=anomalies.index,
            y=anomalies['price'],
            mode='markers',
            name='Anomalía de Predicción',
            marker=dict(symbol='circle', color='red', size=10),
            hovertext=[f"Fecha: {d.strftime('%Y-%m-%d')}<br>Precio: ${p:.2f}" for d, p in zip(anomalies.index, anomalies['price'])]
        ))

    # Configurar el diseño
    fig.update_layout(
        title='Detección de Anomalías Futuras con LSTM',
        xaxis_title='Fecha',
        yaxis_title='Precio',
        template='plotly_dark',
        hovermode='x unified'
    )
    
    fig.show()

def generate_recommendation(anomalies, data_df):
    """
    Genera una recomendación de trading basada en la última anomalía detectada
    y los indicadores del último día.

    Parámetros:
    anomalies (pd.DataFrame): DataFrame con los puntos anómalos.
    data_df (pd.DataFrame): DataFrame con todos los datos.

    Retorna:
    tuple: (str, str) con el resumen y la recomendación.
    """
    if anomalies.empty:
        summary = "El análisis no ha detectado anomalías significativas en los datos recientes."
        recommendation = "MANTENER"
        reason = "El mercado parece estable. No se han identificado movimientos de precio inusuales."
        return summary, recommendation, reason

    latest_anomaly = anomalies.iloc[-1]
    last_data = data_df.iloc[-1]

    # Lógica mejorada que considera la dirección de la anomalía y el volumen
    summary = f"El modelo ha detectado una anomalía reciente el {latest_anomaly.name.strftime('%Y-%m-%d')}."
    
    # Evaluar la señal de trading más reciente
    if last_data['MACD'] > last_data['MACD_Signal'] and last_data['MACD'] > 0:
        recommendation = "COMPRAR"
        reason = "La línea MACD ha cruzado por encima de la línea de señal y está en territorio positivo, lo que indica un fuerte impulso alcista."
    elif last_data['MACD'] < last_data['MACD_Signal'] and last_data['MACD'] < 0:
        recommendation = "VENDER"
        reason = "La línea MACD ha cruzado por debajo de la línea de señal y está en territorio negativo, lo que indica un fuerte impulso bajista."
    else:
        recommendation = "MANTENER"
        reason = "Los indicadores no muestran una señal clara. El mercado parece estar en consolidación."

    return summary, recommendation, reason

def run_backtest(df, strategy_type, ml_model=None):
    """
    Simula una estrategia de trading. Puede ser basada en el cruce de MACD, en un modelo de ML o combinada.
    
    Parámetros:
    df (pd.DataFrame): DataFrame con los datos y los indicadores.
    strategy_type (str): 'MACD', 'ML' o 'Combined'.
    ml_model (RandomForestClassifier): El modelo de ML para la estrategia 'ML' o 'Combined'.

    Retorna:
    pd.DataFrame: DataFrame con los resultados del backtest.
    """
    print(f"\n--- Ejecutando Backtest con Estrategia: {strategy_type} ---")
    portfolio = pd.DataFrame(index=df.index)
    portfolio['asset_price'] = df['price']
    portfolio['holdings'] = 0.0
    portfolio['cash'] = BACKTEST_CONFIG['initial_cash']
    portfolio['total_value'] = portfolio['cash']
    
    in_position = False
    buy_price = 0
    trades = []
    
    features = ['RSI', 'SMA', 'ATR', 'MACD', 'MACD_Signal', 'MACD_Hist', 'BBL', 'BBM', 'BBU']
    
    for i in range(1, len(portfolio)):
        portfolio.loc[portfolio.index[i], 'cash'] = portfolio.loc[portfolio.index[i-1], 'cash']
        portfolio.loc[portfolio.index[i], 'holdings'] = portfolio.loc[portfolio.index[i-1], 'holdings']
        
        signal = 0 # 0 = Mantener, 1 = Comprar, -1 = Vender
        
        # Lógica de señales
        macd_buy_signal = df['MACD'].iloc[i] > df['MACD_Signal'].iloc[i] and df['MACD'].iloc[i-1] <= df['MACD_Signal'].iloc[i-1]
        macd_sell_signal = df['MACD'].iloc[i] < df['MACD_Signal'].iloc[i] and df['MACD'].iloc[i-1] >= df['MACD_Signal'].iloc[i-1]
        
        ml_prediction = None
        if strategy_type in ['ML', 'Combined'] and ml_model is not None and i >= 10:
            data_point = df[features].iloc[i].values.reshape(1, -1)
            ml_prediction = ml_model.predict(data_point)[0]

        if strategy_type == 'MACD':
            if macd_buy_signal:
                signal = 1
            elif macd_sell_signal:
                signal = -1
        
        elif strategy_type == 'ML':
            if ml_prediction is not None:
                if ml_prediction == 1:
                    signal = 1
                elif ml_prediction == 0:
                    signal = -1
        
        elif strategy_type == 'Combined':
            if macd_buy_signal and ml_prediction == 1:
                signal = 1
            elif macd_sell_signal and ml_prediction == 0:
                signal = -1
        
        # Ejecución de la estrategia
        # Comprar
        if signal == 1 and not in_position:
            if portfolio.loc[portfolio.index[i], 'cash'] > 0:
                shares_to_buy = (portfolio.loc[portfolio.index[i], 'cash'] / portfolio.loc[portfolio.index[i], 'asset_price']) * (1 - BACKTEST_CONFIG['transaction_cost_percent'])
                portfolio.loc[portfolio.index[i], 'holdings'] += shares_to_buy
                portfolio.loc[portfolio.index[i], 'cash'] = 0.0
                in_position = True
                buy_price = portfolio.loc[portfolio.index[i], 'asset_price']
                trades.append({'date': portfolio.index[i], 'type': 'buy', 'price': buy_price})
                print(f"COMPRA: el {df.index[i].strftime('%Y-%m-%d')}, precio: ${df['price'].iloc[i]:.2f}")
        
        # Vender
        elif signal == -1 and in_position:
            cash_from_sale = portfolio.loc[portfolio.index[i], 'holdings'] * portfolio.loc[portfolio.index[i], 'asset_price'] * (1 - BACKTEST_CONFIG['transaction_cost_percent'])
            portfolio.loc[portfolio.index[i], 'cash'] = cash_from_sale
            portfolio.loc[portfolio.index[i], 'holdings'] = 0.0
            in_position = False
            trades.append({'date': portfolio.index[i], 'type': 'sell', 'price': portfolio.loc[portfolio.index[i], 'asset_price']})
            print(f"VENTA: el {df.index[i].strftime('%Y-%m-%d')}, precio: ${df['price'].iloc[i]:.2f}")
        
        # Gestión de Riesgos: Stop-Loss
        elif in_position and (portfolio.loc[portfolio.index[i], 'asset_price'] / buy_price - 1) < -BACKTEST_CONFIG['stop_loss_percent']:
            print(f"STOP-LOSS ACTIVADO: el {df.index[i].strftime('%Y-%m-%d')}, precio: ${df['price'].iloc[i]:.2f}")
            cash_from_sale = portfolio.loc[portfolio.index[i], 'holdings'] * portfolio.loc[portfolio.index[i], 'asset_price'] * (1 - BACKTEST_CONFIG['transaction_cost_percent'])
            portfolio.loc[portfolio.index[i], 'cash'] = cash_from_sale
            portfolio.loc[portfolio.index[i], 'holdings'] = 0.0
            in_position = False
            trades.append({'date': portfolio.index[i], 'type': 'stop_loss', 'price': portfolio.loc[portfolio.index[i], 'asset_price']})
            
        portfolio.loc[portfolio.index[i], 'total_value'] = portfolio.loc[portfolio.index[i], 'cash'] + portfolio.loc[portfolio.index[i], 'holdings'] * portfolio.loc[portfolio.index[i], 'asset_price']

    # Cierre de la última posición si sigue abierta al final
    if in_position:
        cash_from_sale = portfolio.iloc[-1]['holdings'] * portfolio.iloc[-1]['asset_price'] * (1 - BACKTEST_CONFIG['transaction_cost_percent'])
        portfolio.iloc[-1]['cash'] = cash_from_sale
        portfolio.iloc[-1]['holdings'] = 0.0
        portfolio.iloc[-1]['total_value'] = portfolio.iloc[-1]['cash']
        trades.append({'date': portfolio.index[-1], 'type': 'sell', 'price': portfolio.iloc[-1]['asset_price']})

    # Calcular métricas de rendimiento
    initial_value = portfolio['total_value'].iloc[0]
    final_value = portfolio['total_value'].iloc[-1]
    
    returns = portfolio['total_value'].pct_change().dropna()
    sharpe_ratio = np.sqrt(252) * returns.mean() / returns.std() if returns.std() != 0 else 0
    
    portfolio['drawdown'] = (portfolio['total_value'].cummax() - portfolio['total_value'])
    max_drawdown_value = portfolio['drawdown'].max()
    max_drawdown_percent = (max_drawdown_value / portfolio['total_value'].cummax().max()) * 100 if portfolio['total_value'].cummax().max() != 0 else 0

    buy_prices = [trade['price'] for trade in trades if trade['type'] == 'buy']
    sell_prices = [trade['price'] for trade in trades if trade['type'] in ['sell', 'stop_loss']]
    
    winning_trades = sum(1 for i in range(len(buy_prices)) if i < len(sell_prices) and sell_prices[i] > buy_prices[i])
    total_trades = min(len(buy_prices), len(sell_prices))
    win_rate = (winning_trades / total_trades) * 100 if total_trades > 0 else 0

    backtest_return_percent = ((final_value - initial_value) / initial_value) * 100
    market_return_percent = ((df['price'].iloc[-1] - df['price'].iloc[0]) / df['price'].iloc[0]) * 100
    
    days_in_backtest = (portfolio.index[-1] - portfolio.index[0]).days
    annualized_return_percent = ((1 + backtest_return_percent / 100) ** (365 / days_in_backtest) - 1) * 100 if days_in_backtest > 0 else 0

    print("\n--- Resultados del Backtest ---")
    print(f"Valor inicial del portafolio: ${initial_value:.2f}")
    print(f"Valor final del portafolio: ${final_value:.2f}")
    print(f"Retorno del Backtest: {backtest_return_percent:.2f}%")
    print(f"Retorno Anualizado: {annualized_return_percent:.2f}%")
    print(f"Retorno del mercado (Buy & Hold): {market_return_percent:.2f}%")
    print(f"Sharpe Ratio: {sharpe_ratio:.2f}")
    print(f"Máximo Drawdown: {max_drawdown_percent:.2f}%")
    print(f"Tasa de Éxito (Win Rate): {win_rate:.2f}%")
    print(f"Total de operaciones: {total_trades}")
    print("-----------------------------------")


    plot_backtest_interactive(portfolio, df, trades, strategy_type, initial_value)

    return portfolio

def plot_backtest_interactive(portfolio, df, trades, strategy_type, initial_value):
    """
    Visualiza el rendimiento del backtest vs. el mercado de forma interactiva.

    Parámetros:
    portfolio (pd.DataFrame): DataFrame con el valor del portafolio.
    df (pd.DataFrame): DataFrame original con el precio del activo.
    trades (list): Lista de diccionarios de operaciones de trading.
    strategy_type (str): El tipo de estrategia.
    initial_value (float): El valor inicial del portafolio.
    """
    
    fig = go.Figure()

    # Añadir el rastro del valor del portafolio
    fig.add_trace(go.Scatter(
        x=portfolio.index,
        y=portfolio['total_value'],
        mode='lines',
        name='Valor del Portafolio',
        line=dict(color='purple')
    ))

    # Añadir el rastro del rendimiento del mercado
    fig.add_trace(go.Scatter(
        x=df.index,
        y=(df['price'] / df['price'].iloc[0]) * initial_value,
        mode='lines',
        name='Rendimiento del Mercado',
        line=dict(color='orange', dash='dash')
    ))

    # Añadir los marcadores de operaciones
    buy_dates = [t['date'] for t in trades if t['type'] == 'buy']
    buy_prices = [t['price'] for t in trades if t['type'] == 'buy']
    sell_dates = [t['date'] for t in trades if t['type'] == 'sell']
    sell_prices = [t['price'] for t in trades if t['type'] == 'sell']
    sl_dates = [t['date'] for t in trades if t['type'] == 'stop_loss']
    sl_prices = [t['price'] for t in trades if t['type'] == 'stop_loss']

    fig.add_trace(go.Scatter(
        x=buy_dates,
        y=buy_prices,
        mode='markers',
        name='Compra',
        marker=dict(symbol='triangle-up', color='green', size=10),
        hovertext=[f"Compra<br>Fecha: {d.strftime('%Y-%m-%d')}<br>Precio: ${p:.2f}" for d, p in zip(buy_dates, buy_prices)],
        hoverinfo='text'
    ))

    fig.add_trace(go.Scatter(
        x=sell_dates,
        y=sell_prices,
        mode='markers',
        name='Venta',
        marker=dict(symbol='triangle-down', color='red', size=10),
        hovertext=[f"Venta<br>Fecha: {d.strftime('%Y-%m-%d')}<br>Precio: ${p:.2f}" for d, p in zip(sell_dates, sell_prices)],
        hoverinfo='text'
    ))
    
    fig.add_trace(go.Scatter(
        x=sl_dates,
        y=sl_prices,
        mode='markers',
        name='Stop-Loss',
        marker=dict(symbol='x', color='darkred', size=10),
        hovertext=[f"Stop-Loss<br>Fecha: {d.strftime('%Y-%m-%d')}<br>Precio: ${p:.2f}" for d, p in zip(sl_dates, sl_prices)],
        hoverinfo='text'
    ))

    # Configurar el diseño
    fig.update_layout(
        title=f'Rendimiento del Backtest vs. Mercado ({strategy_type} Strategy)',
        xaxis_title='Fecha',
        yaxis_title='Valor del Portafolio ($)',
        template='plotly_dark',
        hovermode='x unified'
    )

    fig.show()

def export_to_excel(df, anomalies, backtest_results, ticker, model_type):
    """
    Exporta el DataFrame completo, un resumen de las anomalías y los resultados del backtest
    a un archivo de Excel.

    Parámetros:
    df (pd.DataFrame): DataFrame con todos los datos.
    anomalies (pd.DataFrame): DataFrame con las anomalías detectadas.
    backtest_results (pd.DataFrame): DataFrame con los resultados del backtest.
    ticker (str): Ticker del activo.
    model_type (str): El modelo utilizado.
    """
    try:
        filename = f"Analisis_Anomalias_{ticker}_{model_type}.xlsx"
        writer = pd.ExcelWriter(filename, engine='xlsxwriter')
        
        df.to_excel(writer, sheet_name='Datos Completos')
        
        if not anomalies.empty:
            anomalies.to_excel(writer, sheet_name='Anomalias Detectadas')
        else:
            empty_df = pd.DataFrame(["No se detectaron anomalías."], columns=["Resultado"])
            empty_df.to_excel(writer, sheet_name='Anomalias Detectadas')
        
        if not backtest_results.empty:
            backtest_results.to_excel(writer, sheet_name='Resultados Backtest')
        
        writer._save()
        print(f"\n¡Análisis exportado a '{filename}' con éxito!")
    except Exception as e:
        print(f"Error al exportar a Excel: {e}")

def main():
    """
    Función principal para ejecutar el detector de anomalías.
    """
    ticker = input("Introduce el ticker de la acción o criptomoneda (ej. AAPL, BTC-USD) según Yahoo Finance. Si la descarga falla, buscaremos automáticamente en otra fuente: ").upper()
    
    data_df = get_real_data(ticker)
    
    if data_df is not None and not data_df.empty:
        print("\n--- Modelos de Detección de Anomalías ---")
        print("1. Isolation Forest (modelo más simple y rápido)")
        print("2. Autoencoder (modelo de IA avanzado)")
        print("3. LSTM (para predicción y anomalías futuras)")
        print("4. One-Class SVM (para patrones complejos)")
        model_choice = input("Elige el modelo a usar (1, 2, 3 o 4): ")
        
        anomalies = pd.DataFrame() # DataFrame vacío por defecto
        
        if model_choice == '1':
            model_type = 'IsolationForest'
            model = optimize_hyperparameters(model_type, data_df)
            predictions = model.fit_predict(data_df)
            data_df[model_type] = predictions
            anomalies = data_df[data_df[model_type] == -1]
            print(f"Número de anomalías detectadas: {(predictions == -1).sum()}")
            plot_interactive_analysis(data_df, anomalies, model_type)
        
        elif model_choice == '2':
            model_type = 'Autoencoder'
            scaler = MinMaxScaler()
            scaled_data = scaler.fit_transform(data_df)
            
            autoencoder = train_autoencoder(scaled_data)
            
            reconstructions = autoencoder.predict(scaled_data)
            mse = np.mean(np.power(scaled_data - reconstructions, 2), axis=1)
            
            threshold = np.mean(mse) + 2 * np.std(mse)
            data_df['anomaly'] = [True if error > threshold else False for error in mse]
            anomalies = data_df[data_df['anomaly']]
            print(f"Número de anomalías detectadas: {anomalies.shape[0]}")
            plot_interactive_analysis(data_df, anomalies, model_type)

        elif model_choice == '3':
            model_type = 'LSTM'
            sequence_length = 30
            scaler = MinMaxScaler()
            scaled_data = scaler.fit_transform(data_df)
            X, y = create_sequences(scaled_data, sequence_length)
            
            test_size = int(len(X) * 0.2)
            X_train, y_train = X[:-test_size], y[:-test_size]
            X_test, y_test = X[-test_size:], y[-test_size:]
            
            lstm_model = train_lstm_model(X_train, y_train)
            
            predictions = lstm_model.predict(X_test)
            
            data_df_plot = data_df.iloc[len(data_df) - len(predictions):].copy()
            
            num_features = data_df.shape[1]
            predictions_denormalized = scaler.inverse_transform(np.c_[predictions, np.zeros((len(predictions), num_features - 1))])[:, 0]
            data_df_plot['predicted_price'] = predictions_denormalized
            data_df_plot['prediction_error'] = np.abs(data_df_plot['price'] - data_df_plot['predicted_price'])
            
            threshold = np.mean(data_df_plot['prediction_error']) + 2 * np.std(data_df_plot['prediction_error'])
            data_df_plot['anomaly'] = data_df_plot['prediction_error'] > threshold
            anomalies = data_df_plot[data_df_plot['anomaly']]
            print(f"Número de anomalías de predicción detectadas: {anomalies.shape[0]}")
            plot_lstm_anomalies_interactive(data_df, predictions, scaler)

        elif model_choice == '4':
            model_type = 'OneClassSVM'
            model = optimize_hyperparameters(model_type, data_df)
            scaler = MinMaxScaler()
            scaled_data = scaler.fit_transform(data_df)
            predictions = model.fit_predict(scaled_data)
            data_df[model_type] = predictions
            anomalies = data_df[data_df[model_type] == -1]
            print(f"Número de anomalías detectadas: {(predictions == -1).sum()}")
            plot_interactive_analysis(data_df, anomalies, model_type)

        else:
            print("Elección de modelo inválida. Por favor, reinicia el script.")
            sys.exit()

        print("\n--- Resumen del Análisis ---")
        summary, recommendation, reason = generate_recommendation(anomalies, data_df)
        print(f"Resumen: {summary}")
        print(f"Recomendación: {recommendation}")
        print(f"Motivo: {reason}")
        print("\n*Este análisis es solo para fines informativos y no debe ser considerado un consejo financiero.*")
        
        backtest_results = pd.DataFrame()
        backtest_choice = input("\n¿Deseas realizar un backtest de la estrategia? (s/n): ").lower()
        if backtest_choice == 's':
            print("\n--- Opciones de Backtesting ---")
            print("1. Estrategia de Cruce MACD")
            print("2. Estrategia basada en Machine Learning (Random Forest)")
            print("3. Estrategia Combinada (MACD + ML)")
            strategy_choice = input("Elige la estrategia para el backtest (1, 2 o 3): ")
            
            if strategy_choice == '1':
                backtest_results = run_backtest(data_df, 'MACD')
            elif strategy_choice == '2':
                ml_model = train_ml_model(data_df.copy(), 'RandomForest')
                df_for_backtest = data_df.drop(columns=['target'], errors='ignore')
                backtest_results = run_backtest(df_for_backtest, 'ML', ml_model)
            elif strategy_choice == '3':
                ml_model = train_ml_model(data_df.copy(), 'RandomForest')
                df_for_backtest = data_df.drop(columns=['target'], errors='ignore')
                backtest_results = run_backtest(df_for_backtest, 'Combined', ml_model)
            else:
                print("Elección de estrategia inválida. No se realizará el backtest.")

        export_choice = input("\n¿Deseas exportar los resultados a un archivo de Excel? (s/n): ").lower()
        if export_choice == 's':
            export_to_excel(data_df, anomalies, backtest_results, ticker, model_type)
        else:
            print("Análisis terminado. No se ha exportado ningún archivo.")

    else:
        print("No se pudo obtener datos válidos para el análisis.")

if __name__ == "__main__":
    main()
